# Multi-Head Attention Robustness: Final Models Demo

This notebook demonstrates how to load and use the final trained models from the paper:
- **"Inherent Robustness of Multi-Head Attention in Cross-Sectional Asset Pricing: Theory and Empirical Evidence from Finance-Valid Adversarial Attacks"**

## Reproducibility

**Random seed is set to 42** for reproducible results. All models are evaluated in deterministic mode (dropout disabled). Results should be identical across runs.

## Data Splitting

This notebook uses the **same data splitting logic** as the evaluation script (`scripts/evaluate_adversarial_models.py`):
- **Training period**: 2005-01-01 to 2017-12-31
- **Validation period**: 2018-01-01 to 2019-12-31
- **Data preprocessing**: Matches the `CrossSectionalDataSplitter` class from the evaluation script

## Models Available

1. **Linear Baselines**: OLS, Ridge
2. **Neural Baselines**: MLP
3. **Transformer Models**: Single-Head, Multi-Head, Multi-Head Diversity
4. **Adversarially Trained Models**: Models trained with A1, A2, A3 attacks at various epsilons

## Workflow

1. Load data
2. **Train models from scratch** (OLS, Ridge, MLP, Single-Head, Multi-Head, Multi-Head Diversity)
3. **Adversarial training** for transformer models (A1-A4 attacks)
4. Compare standard vs adversarially trained models
5. Make predictions on validation set
6. Evaluate model performance (RMSE, R²)
7. Visualize predictions and training curves

In [4]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"Set random seed to {RANDOM_SEED} for reproducibility")

# Add parent directory to path
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(repo_root))

# Import model definitions
from src.models.feature_token_transformer import FeatureTokenTransformer, SingleHeadTransformer

print(f"Working directory: {Path.cwd()}")
print(f"Repository root: {repo_root}")

Set random seed to 42 for reproducibility
Working directory: /Users/zelalemabahana/multihead-attention-robustness/notebooks
Repository root: /Users/zelalemabahana/multihead-attention-robustness


## 1. Load Data

In [5]:
# Load cross-sectional data
data_path = repo_root / 'data' / 'cross_sectional' / 'master_table.csv'
print(f"Loading data from: {data_path}")

df = pd.read_csv(data_path)
print(f"Data shape: {df.shape}")
print(f"Columns: {list(df.columns[:10])}...")

# Set date as index for proper time-series splitting (matching evaluation script)
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    print(f"\nDate range: {df.index.min()} to {df.index.max()}")

df.head()

Loading data from: /Users/zelalemabahana/multihead-attention-robustness/data/cross_sectional/master_table.csv
Data shape: (31534, 27)
Columns: ['date', 'symbol', 'mom_1m', 'mom_6m', 'mom_12m', 'mom_12_1m', 'vol_3m', 'vol_12m', 'price', 'log_price']...

Date range: 2005-01-31 00:00:00 to 2025-12-31 00:00:00


,symbol,mom_1m,mom_6m,mom_12m,mom_12_1m,vol_3m,vol_12m,price,log_price,returns_1d,...,pb_ratio,dividend_yield,eps,roe,profit_margin,revenue_per_share,market_cap,covid_period,ret_fwd_1m,mktcap
date,,,,,,,,,,,,,,,,,,,,,
2005-01-31,AAP,-0.007141,0.161412,0.107400,0.114541,0.177014,0.272406,24.240944,3.188043,0.012926,...,1.126753,2.42,-10.19,-0.23868,-0.04369,144.046,2.474117e+09,0,0.168909,2.474117e+09
2005-01-31,ABCB,-0.054833,0.176429,0.130269,0.185102,0.303383,0.286354,12.617585,2.535091,0.020060,...,1.307446,1.04,5.77,0.10345,0.35227,16.466,5.272271e+09,0,0.057030,5.272271e+09
2005-01-31,AEO,0.073315,0.543977,1.736371,1.663056,0.253181,0.316677,8.883611,2.184208,0.026263,...,2.833750,1.84,1.13,0.12362,0.03903,30.098,4.605641e+09,0,0.065551,4.605641e+09
2005-01-31,ALGN,-0.201843,-0.489387,-0.582651,-0.380807,0.446573,0.566262,8.660000,2.158715,-0.011416,...,3.036553,NaN,5.14,0.09577,0.09501,54.387,1.209145e+10,0,-0.125866,1.209145e+10
2005-01-31,AMAT,-0.066902,-0.067995,-0.266944,-0.200042,0.288008,0.323094,11.517730,2.443888,-0.008729,...,11.350218,0.62,8.67,0.35508,0.24669,35.284,2.327789e+11,0,0.097484,2.327789e+11


In [6]:
# Use the same CrossSectionalDataSplitter as the evaluation script
class CrossSectionalDataSplitter:
    """Simple data splitter for cross-sectional data (matching evaluation script)."""
    
    def __init__(self, train_start='2005-01-01', train_end='2017-12-31',
                 val_start='2018-01-01', val_end='2019-12-31'):
        self.train_start = train_start
        self.train_end = train_end
        self.val_start = val_start
        self.val_end = val_end
    
    def split(self, master_table):
        """Split data into train and validation sets."""
        master_table.index = pd.to_datetime(master_table.index)
        
        train_data = master_table.loc[self.train_start:self.train_end]
        val_data = master_table.loc[self.val_start:self.val_end]
        
        return {
            'train': train_data,
            'val': val_data
        }
    
    def prepare_features_labels(self, data):
        """Prepare features and labels from data (matching evaluation script logic)."""
        if data.empty:
            return pd.DataFrame(), pd.Series()
        
        numeric_data = data.select_dtypes(include=[np.number])
        
        if numeric_data.empty:
            print("Warning: No numeric columns found in data")
            return pd.DataFrame(), pd.Series()
        
        exclude_cols = ['mktcap', 'market_cap', 'date', 'year', 'month', 'ticker', 'permno', 'gvkey']
        target_cols = ['return', 'returns', 'ret', 'target', 'y', 'next_return', 'forward_return', 
                      'ret_1', 'ret_1m', 'ret_12m', 'future_return', 'returns_1d']
        
        target_col = None
        for tc in target_cols:
            for col in numeric_data.columns:
                if tc.lower() in col.lower() and col.lower() not in [ec.lower() for ec in exclude_cols]:
                    target_col = col
                    break
            if target_col:
                break
        
        if target_col is None:
            potential_targets = [col for col in numeric_data.columns 
                               if col.lower() not in [ec.lower() for ec in exclude_cols]]
            if potential_targets:
                target_col = potential_targets[-2] if len(potential_targets) > 1 else potential_targets[-1]
            else:
                target_col = numeric_data.columns[-1]
        
        feature_cols = [col for col in numeric_data.columns 
                       if col != target_col and col.lower() not in [ec.lower() for ec in exclude_cols]]
        
        if not feature_cols:
            feature_cols = [col for col in numeric_data.columns if col != target_col]
        
        if not feature_cols:
            feature_cols = numeric_data.columns[:-1].tolist()
            target_col = numeric_data.columns[-1]
        
        X = numeric_data[feature_cols]
        y = numeric_data[target_col]
        
        return X, y

# Initialize splitter and split data
splitter = CrossSectionalDataSplitter()
data_splits = splitter.split(df)

train_df = data_splits['train']
val_df = data_splits['val']

print(f"Train period: {splitter.train_start} to {splitter.train_end}")
print(f"Validation period: {splitter.val_start} to {splitter.val_end}")
print(f"Train set: {train_df.shape[0]} samples")
print(f"Validation set: {val_df.shape[0]} samples")

# Prepare features and labels using the same logic as evaluation script
X_train_df, y_train = splitter.prepare_features_labels(train_df)
X_val_df, y_val = splitter.prepare_features_labels(val_df)

print(f"\nNumber of features: {X_train_df.shape[1]}")
print(f"Target column: {y_train.name}")

# Fill NaN values and convert to numpy arrays
X_train = X_train_df.fillna(0).values.astype(np.float32)
y_train = y_train.fillna(0).values.astype(np.float32)
X_val = X_val_df.fillna(0).values.astype(np.float32)
y_val = y_val.fillna(0).values.astype(np.float32)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(f"\nTrain features shape: {X_train_scaled.shape}")
print(f"Validation features shape: {X_val_scaled.shape}")
print(f"Feature columns: {list(X_train_df.columns[:5])}... ({len(X_train_df.columns)} total)")

Train period: 2005-01-01 to 2017-12-31
Validation period: 2018-01-01 to 2019-12-31
Train set: 18826 samples
Validation set: 3408 samples

Number of features: 22
Target column: returns_1d

Train features shape: (18826, 22)
Validation features shape: (3408, 22)
Feature columns: ['mom_1m', 'mom_6m', 'mom_12m', 'mom_12_1m', 'vol_3m']... (22 total)


## 2. Train Models

Train all models from scratch on the training data.

In [7]:
# Training configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Store trained models
models = {}
training_history = {}

# Training hyperparameters
TRAINING_CONFIG = {
    'ols': {},
    'ridge': {'alpha': 1.0},
    'mlp': {
        'hidden_dims': [128, 64],
        'learning_rate': 0.001,
        'batch_size': 64,
        'epochs': 100,
        'patience': 10
    },
    'transformer': {
        'd_model': 72,
        'num_heads': 8,  
        'num_layers': 2,
        'd_ff': 512,
        'dropout': 0.1,
        'learning_rate': 0.0001,
        'batch_size': 32,
        'epochs': 100,
        'patience': 20
    }
}

Using device: cpu


In [8]:
# Training configurationdevice = 'cuda' if torch.cuda.is_available() else 'cpu'print(f"Using device: {device}")# Store trained modelsmodels = {}training_history = {}# Training hyperparametersTRAINING_CONFIG = {    'ols': {},    'ridge': {'alpha': 1.0},    'mlp': {        'hidden_dims': [128, 64],        'learning_rate': 0.001,        'batch_size': 64,        'epochs': 50,        'patience': 10    },    'transformer': {        'd_model': 64,        'num_heads': 6,        'num_layers': 2,        'd_ff': 512,        'dropout': 0.1,        'learning_rate': 0.0001,        'batch_size': 32,        'epochs': 50,        'patience': 20    }}

In [9]:
# Train OLS (Linear Regression)
from sklearn.linear_model import LinearRegression

print("=" * 80)
print("TRAINING OLS MODEL")
print("=" * 80)

ols_model = LinearRegression()
ols_model.fit(X_train_scaled, y_train)

models['OLS'] = ols_model

# Evaluate on validation set
ols_pred = ols_model.predict(X_val_scaled)
ols_rmse = np.sqrt(mean_squared_error(y_val, ols_pred))
ols_r2 = r2_score(y_val, ols_pred)

print(f"✓ OLS trained")
print(f"  Validation RMSE: {ols_rmse:.6f}")
print(f"  Validation R²: {ols_r2:.6f}")

training_history['OLS'] = {'rmse': ols_rmse, 'r2': ols_r2}

TRAINING OLS MODEL
✓ OLS trained
  Validation RMSE: 0.017520
  Validation R²: -0.007658


In [10]:
# Train Ridge Regression
from sklearn.linear_model import Ridge

print("=" * 80)
print("TRAINING RIDGE MODEL")
print("=" * 80)

ridge_model = Ridge(alpha=TRAINING_CONFIG['ridge']['alpha'], random_state=RANDOM_SEED)
ridge_model.fit(X_train_scaled, y_train)

models['Ridge'] = ridge_model

# Evaluate on validation set
ridge_pred = ridge_model.predict(X_val_scaled)
ridge_rmse = np.sqrt(mean_squared_error(y_val, ridge_pred))
ridge_r2 = r2_score(y_val, ridge_pred)

print(f"✓ Ridge trained")
print(f"  Validation RMSE: {ridge_rmse:.6f}")
print(f"  Validation R²: {ridge_r2:.6f}")

training_history['Ridge'] = {'rmse': ridge_rmse, 'r2': ridge_r2}

TRAINING RIDGE MODEL
✓ Ridge trained
  Validation RMSE: 0.017519
  Validation R²: -0.007585


In [11]:
# Train MLP (Multi-Layer Perceptron)
print("=" * 80)
print("TRAINING MLP MODEL")
print("=" * 80)

class MLP(nn.Module):
    """Simple MLP for regression."""
    def __init__(self, input_dim, hidden_dims, dropout=0.1):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x).squeeze(-1)

mlp_config = TRAINING_CONFIG['mlp']
mlp_model = MLP(
    input_dim=X_train_scaled.shape[1],
    hidden_dims=mlp_config['hidden_dims'],
    dropout=0.1
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=mlp_config['learning_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Training loop
best_val_loss = float('inf')
patience_counter = 0
train_losses = []
val_losses = []

X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)

for epoch in range(mlp_config['epochs']):
    # Training
    mlp_model.train()
    optimizer.zero_grad()
    train_pred = mlp_model(X_train_tensor)
    train_loss = criterion(train_pred, y_train_tensor)
    train_loss.backward()
    torch.nn.utils.clip_grad_norm_(mlp_model.parameters(), max_norm=1.0)
    optimizer.step()
    
    # Validation
    mlp_model.eval()
    with torch.no_grad():
        val_pred = mlp_model(X_val_tensor)
        val_loss = criterion(val_pred, y_val_tensor)
    
    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    scheduler.step(val_loss)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= mlp_config['patience']:
            print(f"  Early stopping at epoch {epoch+1}")
            break
    
    if (epoch + 1) % 10 == 0:
        print(f"  Epoch {epoch+1}/{mlp_config['epochs']}: Train Loss={train_loss.item():.6f}, Val Loss={val_loss.item():.6f}")

mlp_model.eval()
with torch.no_grad():
    mlp_pred = mlp_model(X_val_tensor).cpu().numpy()

mlp_rmse = np.sqrt(mean_squared_error(y_val, mlp_pred))
mlp_r2 = r2_score(y_val, mlp_pred)

models['MLP'] = mlp_model
print(f"✓ MLP trained")
print(f"  Validation RMSE: {mlp_rmse:.6f}")
print(f"  Validation R²: {mlp_r2:.6f}")

training_history['MLP'] = {'rmse': mlp_rmse, 'r2': mlp_r2, 'train_losses': train_losses, 'val_losses': val_losses}

TRAINING MLP MODEL
  Epoch 10/100: Train Loss=0.003232, Val Loss=0.001477
  Epoch 20/100: Train Loss=0.002140, Val Loss=0.000649
  Epoch 30/100: Train Loss=0.001559, Val Loss=0.000554
  Epoch 40/100: Train Loss=0.001308, Val Loss=0.000499
  Epoch 50/100: Train Loss=0.001119, Val Loss=0.000447
  Epoch 60/100: Train Loss=0.001020, Val Loss=0.000411
  Epoch 70/100: Train Loss=0.000936, Val Loss=0.000399
  Epoch 80/100: Train Loss=0.000861, Val Loss=0.000381
  Epoch 90/100: Train Loss=0.000810, Val Loss=0.000374
  Epoch 100/100: Train Loss=0.000769, Val Loss=0.000367
✓ MLP trained
  Validation RMSE: 0.019169
  Validation R²: -0.206334


In [12]:
# Train Transformer Models
print("=" * 80)
print("TRAINING TRANSFORMER MODELS")
print("=" * 80)

def train_transformer(model, model_name, X_train, y_train, X_val, y_val, config, device='cpu'):
    """Train a transformer model."""
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    
    # Handle feature dimension mismatch
    num_features = model.num_features if hasattr(model, 'num_features') else model.model.num_features
    
    if X_train.shape[1] != num_features:
        if X_train.shape[1] < num_features:
            # Pad
            padding_train = np.zeros((X_train.shape[0], num_features - X_train.shape[1]))
            padding_val = np.zeros((X_val.shape[0], num_features - X_val.shape[1]))
            X_train_tensor = torch.FloatTensor(np.hstack([X_train, padding_train])).to(device)
            X_val_tensor = torch.FloatTensor(np.hstack([X_val, padding_val])).to(device)
        else:
            # Truncate
            X_train_tensor = torch.FloatTensor(X_train[:, :num_features]).to(device)
            X_val_tensor = torch.FloatTensor(X_val[:, :num_features]).to(device)
    
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    
    batch_size = config['batch_size']
    n_batches = (len(X_train_tensor) + batch_size - 1) // batch_size
    
    for epoch in range(config['epochs']):
        # Training
        model.train()
        epoch_train_loss = 0.0
        
        for i in range(0, len(X_train_tensor), batch_size):
            batch_X = X_train_tensor[i:i+batch_size]
            batch_y = y_train_tensor[i:i+batch_size]
            
            optimizer.zero_grad()
            pred = model(batch_X)
            if isinstance(pred, tuple):
                pred = pred[0]
            loss = criterion(pred.squeeze(), batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_train_loss += loss.item()
        
        epoch_train_loss /= n_batches
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_pred = model(X_val_tensor)
            if isinstance(val_pred, tuple):
                val_pred = val_pred[0]
            val_loss = criterion(val_pred.squeeze(), y_val_tensor)
        
        train_losses.append(epoch_train_loss)
        val_losses.append(val_loss.item())
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print(f"  {model_name}: Early stopping at epoch {epoch+1}")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"  {model_name} - Epoch {epoch+1}/{config['epochs']}: Train Loss={epoch_train_loss:.6f}, Val Loss={val_loss.item():.6f}")
    
    # Final evaluation
    model.eval()
    with torch.no_grad():
        final_pred = model(X_val_tensor)
        if isinstance(final_pred, tuple):
            final_pred = final_pred[0]
        final_pred = final_pred.squeeze().cpu().numpy()
    
    return model, final_pred, train_losses, val_losses

# Train Single-Head Transformer
print("\nTraining Single-Head Transformer...")
single_head_model = SingleHeadTransformer(
    num_features=X_train_scaled.shape[1],
    d_model=TRAINING_CONFIG['transformer']['d_model'],
    num_layers=TRAINING_CONFIG['transformer']['num_layers']
)
single_head_model, single_head_pred, sh_train_losses, sh_val_losses = train_transformer(
    single_head_model, 'Single-Head', X_train_scaled, y_train, X_val_scaled, y_val,
    TRAINING_CONFIG['transformer'], device
)

single_head_rmse = np.sqrt(mean_squared_error(y_val, single_head_pred))
single_head_r2 = r2_score(y_val, single_head_pred)

models['Single-Head'] = single_head_model
print(f"Single-Head trained - RMSE: {single_head_rmse:.6f}, R²: {single_head_r2:.6f}")
training_history['Single-Head'] = {'rmse': single_head_rmse, 'r2': single_head_r2, 'train_losses': sh_train_losses, 'val_losses': sh_val_losses}

TRAINING TRANSFORMER MODELS

Training Single-Head Transformer...
  Single-Head - Epoch 10/100: Train Loss=0.000595, Val Loss=0.000295
  Single-Head - Epoch 20/100: Train Loss=0.000539, Val Loss=0.000285
  Single-Head - Epoch 30/100: Train Loss=0.000506, Val Loss=0.000282
  Single-Head - Epoch 40/100: Train Loss=0.000468, Val Loss=0.000279
  Single-Head - Epoch 50/100: Train Loss=0.000453, Val Loss=0.000277
  Single-Head - Epoch 60/100: Train Loss=0.000448, Val Loss=0.000276
  Single-Head - Epoch 70/100: Train Loss=0.000443, Val Loss=0.000276
  Single-Head: Early stopping at epoch 79
Single-Head trained - RMSE: 0.016650, R²: 0.089910


In [ ]:
%%time
# Train Multi-Head Transformer
print("\nTraining Multi-Head Transformer...")
multi_head_model = FeatureTokenTransformer(
    num_features=X_train_scaled.shape[1],
    d_model=TRAINING_CONFIG['transformer']['d_model'],
    num_heads=TRAINING_CONFIG['transformer']['num_heads'],
    num_layers=TRAINING_CONFIG['transformer']['num_layers'],
    d_ff=TRAINING_CONFIG['transformer']['d_ff'],
    dropout=TRAINING_CONFIG['transformer']['dropout'],
    use_head_diversity=False
)
multi_head_model, multi_head_pred, mh_train_losses, mh_val_losses = train_transformer(
    multi_head_model, 'Multi-Head', X_train_scaled, y_train, X_val_scaled, y_val,
    TRAINING_CONFIG['transformer'], device
)

multi_head_rmse = np.sqrt(mean_squared_error(y_val, multi_head_pred))
multi_head_r2 = r2_score(y_val, multi_head_pred)

models['Multi-Head'] = multi_head_model
print(f"✓ Multi-Head trained - RMSE: {multi_head_rmse:.6f}, R²: {multi_head_r2:.6f}")
training_history['Multi-Head'] = {'rmse': multi_head_rmse, 'r2': multi_head_r2, 'train_losses': mh_train_losses, 'val_losses': mh_val_losses}


Training Multi-Head Transformer...
  Multi-Head - Epoch 10/100: Train Loss=0.000608, Val Loss=0.000302


In [ ]:
# Train Multi-Head Diversity Transformer
print("\nTraining Multi-Head Diversity Transformer...")
multi_head_diversity_model = FeatureTokenTransformer(
    num_features=X_train_scaled.shape[1],
    d_model=TRAINING_CONFIG['transformer']['d_model'],
    num_heads=TRAINING_CONFIG['transformer']['num_heads'],
    num_layers=TRAINING_CONFIG['transformer']['num_layers'],
    d_ff=TRAINING_CONFIG['transformer']['d_ff'],
    dropout=TRAINING_CONFIG['transformer']['dropout'],
    use_head_diversity=True,
    diversity_weight=0.01
)
multi_head_diversity_model, mhd_pred, mhd_train_losses, mhd_val_losses = train_transformer(
    multi_head_diversity_model, 'Multi-Head Diversity', X_train_scaled, y_train, X_val_scaled, y_val,
    TRAINING_CONFIG['transformer'], device
)

multi_head_diversity_rmse = np.sqrt(mean_squared_error(y_val, mhd_pred))
multi_head_diversity_r2 = r2_score(y_val, mhd_pred)

models['Multi-Head Diversity'] = multi_head_diversity_model
print(f"✓ Multi-Head Diversity trained - RMSE: {multi_head_diversity_rmse:.6f}, R²: {multi_head_diversity_r2:.6f}")
training_history['Multi-Head Diversity'] = {'rmse': multi_head_diversity_rmse, 'r2': multi_head_diversity_r2, 'train_losses': mhd_train_losses, 'val_losses': mhd_val_losses}

print("\n" + "=" * 80)
print("ALL MODELS TRAINED")
print("=" * 80)
print(f"Total models trained: {len(models)}")
print("\nValidation Results Summary:")
for name, metrics in training_history.items():
    print(f"  {name}: RMSE={metrics['rmse']:.6f}, R²={metrics['r2']:.6f}")

In [ ]:
def make_predictions(model, X, device='cpu', batch_size=128, is_sklearn=False):
    """Make predictions with a model (deterministic mode)."""
    if is_sklearn:
        # sklearn models
        return model.predict(X)
    
    # PyTorch models
    model.eval()
    
    # Ensure deterministic mode - disable dropout
    with torch.no_grad():
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.eval()
        
        predictions = []
        
        # Get num_features - handle both FeatureTokenTransformer and SingleHeadTransformer
        if hasattr(model, 'num_features'):
            num_features = model.num_features
        elif hasattr(model, 'model') and hasattr(model.model, 'num_features'):
            # SingleHeadTransformer wraps FeatureTokenTransformer in self.model
            num_features = model.model.num_features
        else:
            # Fallback: use input dimension
            num_features = X.shape[1]
        
        for i in range(0, len(X), batch_size):
            batch = X[i:i+batch_size]
            X_tensor = torch.FloatTensor(batch).to(device)
            
            # Handle padding if needed
            if X_tensor.shape[1] != num_features:
                if X_tensor.shape[1] < num_features:
                    padding = torch.zeros(X_tensor.shape[0], num_features - X_tensor.shape[1]).to(device)
                    X_tensor = torch.cat([X_tensor, padding], dim=1)
                else:
                    X_tensor = X_tensor[:, :num_features]
            
            pred = model(X_tensor)
            if isinstance(pred, tuple):
                pred = pred[0]
            predictions.append(pred.cpu().numpy())
    
    return np.concatenate(predictions, axis=0).flatten()

# Make predictions on validation set
print("Making predictions on validation set...")
predictions = {}

for name, model in models.items():
    is_sklearn = name in ['OLS', 'Ridge']
    pred = make_predictions(model, X_val_scaled, device, is_sklearn=is_sklearn)
    predictions[name] = pred.flatten()
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    r2 = r2_score(y_val, pred)
    
    print(f"{name}:")
    print(f"  RMSE: {rmse:.6f}")
    print(f"  R²: {r2:.6f}")
    print()
comparison_data = []
for name in ['OLS', 'Ridge', 'MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']:
    if name in training_history:
        comparison_data.append({
            'Model': name,
            'RMSE': training_history[name]['rmse'],
            'R²': training_history[name]['r2']
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('R²', ascending=False)

print("=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
ax1.barh(comparison_df['Model'], comparison_df['RMSE'])
ax1.set_xlabel('RMSE', fontsize=12)
ax1.set_title('RMSE Comparison (Lower is Better)', fontsize=14)
ax1.grid(True, alpha=0.3, axis='x')

# R² comparison
ax2.barh(comparison_df['Model'], comparison_df['R²'])
ax2.set_xlabel('R²', fontsize=12)
ax2.set_title('R² Comparison (Higher is Better)', fontsize=14)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 3. Adversarial Training (A1-A4 Attacks)

Train transformer models with adversarial training against A1-A4 attacks to improve robustness.

In [ ]:
# Adversarial Attack Implementations (A1-A4)
def apply_a1_attack(X, epsilon=0.01):
    """A1: Measurement Error - bounded perturbations."""
    noise = np.random.normal(0, epsilon, X.shape)
    # Scale noise by feature standard deviation
    feature_std = np.std(X, axis=0, keepdims=True) + 1e-8
    noise = noise * feature_std
    return X + noise


def apply_a2_attack(X, missing_rate=0.1):
    """A2: Missingness/Staleness - set random features to zero."""
    X_adv = X.copy()
    n_samples, n_features = X.shape
    n_missing = int(n_features * missing_rate)
    
    for i in range(n_samples):
        missing_indices = np.random.choice(n_features, n_missing, replace=False)
        X_adv[i, missing_indices] = 0.0
    
    return X_adv


def apply_a3_attack(X, epsilon=0.01):
    """A3: Rank Manipulation - cross-sectional perturbation preserving ranks."""
    X_adv = X.copy()
    n_samples = X.shape[0]
    
    # Add small random perturbation that preserves relative ordering
    for i in range(n_samples):
        perturbation = np.random.normal(0, epsilon, X.shape[1])
        # Scale by feature std to maintain relative magnitudes
        feature_std = np.std(X[i], axis=0) + 1e-8
        perturbation = perturbation * feature_std
        X_adv[i] = X[i] + perturbation
    
    return X_adv


def apply_a4_attack(X, epsilon=1.0):
    """A4: Regime Shift - distribution shift attack."""
    # A4 simulates regime shift by scaling volatility
    # epsilon acts as volatility multiplier
    X_adv = X.copy()
    feature_std = np.std(X, axis=0, keepdims=True) + 1e-8
    # Generate noise with std = epsilon, then scale by feature std
    noise = np.random.normal(0, epsilon, X.shape) * feature_std
    X_adv = X + noise
    return X_adv





In [ ]:
# Adversarial Training Configuration
ADVERSARIAL_CONFIG = {
    'epsilons': [0.25, 0.5, 1.0],  # Attack strengths
    'attacks': ['a1', 'a2', 'a3', 'a4'],  # Attack types
    'robust_weight': 0.3,  # Weight for adversarial loss (0.3 = 30% adversarial, 70% clean)
    'learning_rate': 0.0001,
    'batch_size': 32,
    'epochs': 100,
    'patience': 20,
    'warmup_epochs': 5  # Gradually increase adversarial weight
}

# Store adversarially trained models
adversarial_models = {}
adversarial_training_history = {}

In [ ]:
def adversarial_training_step(model, X_batch, y_batch, attack_type, epsilon, 
                             optimizer, device='cpu', robust_weight=0.3):
    """
    Perform one adversarial training step.
    
    Args:
        model: The model to train
        X_batch: Input batch (numpy array)
        y_batch: Target batch (numpy array)
        attack_type: 'a1', 'a2', 'a3', or 'a4'
        epsilon: Attack strength
        optimizer: Optimizer
        device: Device to use
        robust_weight: Weight for adversarial loss
    
    Returns:
        Dictionary with loss values or None if batch is invalid
    """
    model.train()
    optimizer.zero_grad()
    
    # Convert to tensors
    X_tensor = torch.FloatTensor(X_batch).to(device)
    y_tensor = torch.FloatTensor(y_batch).to(device)
    
    # Clean forward pass
    output_clean = model(X_tensor)
    if isinstance(output_clean, tuple):
        y_pred_clean = output_clean[0]
    else:
        y_pred_clean = output_clean
    
    # Check for NaN/Inf in predictions
    if torch.any(torch.isnan(y_pred_clean)) or torch.any(torch.isinf(y_pred_clean)):
        return None
    
    clean_loss = nn.MSELoss()(y_pred_clean.squeeze(), y_tensor)
    
    # Check if clean_loss is valid
    if torch.isnan(clean_loss) or torch.isinf(clean_loss):
        return None
    
    # Generate adversarial examples
    if attack_type == 'a1':
        X_adv = apply_a1_attack(X_batch, epsilon=epsilon)
    elif attack_type == 'a2':
        # For A2, epsilon controls missing rate
        missing_rate = min(epsilon / 10.0, 0.8)  # Convert epsilon to missing rate
        X_adv = apply_a2_attack(X_batch, missing_rate=missing_rate)
    elif attack_type == 'a3':
        X_adv = apply_a3_attack(X_batch, epsilon=epsilon)
    elif attack_type == 'a4':
        X_adv = apply_a4_attack(X_batch, epsilon=epsilon)
    else:
        raise ValueError(f"Unknown attack type: {attack_type}")
    
    # Adversarial forward pass
    X_adv_tensor = torch.FloatTensor(X_adv).to(device)
    output_adv = model(X_adv_tensor)
    if isinstance(output_adv, tuple):
        y_pred_adv = output_adv[0]
    else:
        y_pred_adv = output_adv
    
    # Check for NaN/Inf in adversarial predictions
    if torch.any(torch.isnan(y_pred_adv)) or torch.any(torch.isinf(y_pred_adv)):
        return None
    
    adv_loss = nn.MSELoss()(y_pred_adv.squeeze(), y_tensor)
    
    # Check if adv_loss is valid
    if torch.isnan(adv_loss) or torch.isinf(adv_loss):
        return None
    
    # Combined loss
    total_loss = (1 - robust_weight) * clean_loss + robust_weight * adv_loss
    
    # Check if total_loss is valid before backward pass
    if torch.isnan(total_loss) or torch.isinf(total_loss):
        return None
    
    # Ensure total_loss requires gradients
    if not total_loss.requires_grad:
        return None
    
    # Backward pass with error handling
    try:
        total_loss.backward()
    except RuntimeError as e:
        if "does not require grad" in str(e) or "does not have a grad_fn" in str(e):
            optimizer.zero_grad()
            return None
        else:
            raise
    
    # Gradient clipping to prevent exploding gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    
    return {
        'clean_loss': clean_loss.item(),
        'adversarial_loss': adv_loss.item(),
        'total_loss': total_loss.item()
    }

In [ ]:
def train_adversarial_model(model, model_name, X_train, y_train, X_val, y_val, 
                           attack_type, epsilon, config, device='cpu'):
    """
    Train model with adversarial training.
    
    Args:
        model: Model to train (will be copied)
        model_name: Name of the model
        X_train: Training features
        y_train: Training targets
        X_val: Validation features
        y_val: Validation targets
        attack_type: 'a1', 'a2', 'a3', or 'a4'
        epsilon: Attack strength
        config: Training configuration
        device: Device to use
    
    Returns:
        Trained model, predictions, and training history
    """
    # Create a fresh copy of the model for adversarial training
    import copy
    model = copy.deepcopy(model)
    model = model.to(device)
    model.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6
    )
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    
    # Handle feature dimension mismatch
    num_features = model.num_features if hasattr(model, 'num_features') else model.model.num_features
    
    if X_train.shape[1] != num_features:
        if X_train.shape[1] < num_features:
            # Pad
            padding_train = np.zeros((X_train.shape[0], num_features - X_train.shape[1]))
            padding_val = np.zeros((X_val.shape[0], num_features - X_val.shape[1]))
            X_train_tensor = torch.FloatTensor(np.hstack([X_train, padding_train])).to(device)
            X_val_tensor = torch.FloatTensor(np.hstack([X_val, padding_val])).to(device)
        else:
            # Truncate
            X_train_tensor = torch.FloatTensor(X_train[:, :num_features]).to(device)
            X_val_tensor = torch.FloatTensor(X_val[:, :num_features]).to(device)
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_clean_loss': [],
        'train_adv_loss': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    warmup_epochs = config.get('warmup_epochs', 5)
    
    batch_size = config['batch_size']
    n_batches = (len(X_train_tensor) + batch_size - 1) // batch_size
    
    for epoch in range(config['epochs']):
        # Gradual warmup: increase robust_weight from 0.1 to target value
        if epoch < warmup_epochs:
            current_robust_weight = 0.1 + (config['robust_weight'] - 0.1) * (epoch / warmup_epochs)
        else:
            current_robust_weight = config['robust_weight']
        
        epoch_losses = {'clean': [], 'adv': [], 'total': []}
        
        # Training
        model.train()
        for i in range(0, len(X_train_tensor), batch_size):
            batch_X = X_train_tensor[i:i+batch_size].cpu().numpy()
            batch_y = y_train_tensor[i:i+batch_size].cpu().numpy()
            
            losses = adversarial_training_step(
                model, batch_X, batch_y, attack_type, epsilon,
                optimizer, device, current_robust_weight
            )
            
            # Skip batch if None (invalid batch)
            if losses is None:
                continue
            
            # Check for NaN/Inf in losses
            if (np.isnan(losses['total_loss']) or np.isinf(losses['total_loss']) or
                np.isnan(losses['clean_loss']) or np.isinf(losses['clean_loss']) or
                np.isnan(losses['adversarial_loss']) or np.isinf(losses['adversarial_loss'])):
                continue
            
            epoch_losses['clean'].append(losses['clean_loss'])
            epoch_losses['adv'].append(losses['adversarial_loss'])
            epoch_losses['total'].append(losses['total_loss'])
        
        # Skip epoch if all losses are invalid
        if len(epoch_losses['total']) == 0:
            continue
        
        # Validation
        model.eval()
        with torch.no_grad():
            output_val = model(X_val_tensor)
            if isinstance(output_val, tuple):
                y_pred_val = output_val[0]
            else:
                y_pred_val = output_val
            
            # Check for constant predictions (model collapse detection)
            y_pred_np = y_pred_val.squeeze().cpu().numpy()
            pred_std = np.std(y_pred_np)
            
            if pred_std < 1e-8:
                print(f"   MODEL COLLAPSE DETECTED at epoch {epoch+1}!")
                break
            
            val_loss = nn.MSELoss()(y_pred_val.squeeze(), y_val_tensor).item()
            
            # Check for NaN/Inf in validation loss
            if np.isnan(val_loss) or np.isinf(val_loss):
                val_loss = float('inf')
        
        # Record history
        avg_train_loss = np.mean(epoch_losses['total']) if epoch_losses['total'] else float('inf')
        avg_clean_loss = np.mean(epoch_losses['clean']) if epoch_losses['clean'] else 0.0
        avg_adv_loss = np.mean(epoch_losses['adv']) if epoch_losses['adv'] else 0.0
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss)
        history['train_clean_loss'].append(avg_clean_loss)
        history['train_adv_loss'].append(avg_adv_loss)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        # Early stopping
        if not (np.isnan(val_loss) or np.isinf(val_loss)):
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= config['patience']:
                    print(f"  {model_name} ({attack_type.upper()}, ε={epsilon}): Early stopping at epoch {epoch+1}")
                    break
        
        if (epoch + 1) % 10 == 0:
            print(f"  {model_name} ({attack_type.upper()}, ε={epsilon}) - Epoch {epoch+1}/{config['epochs']}: "
                  f"Train Loss={avg_train_loss:.6f}, Val Loss={val_loss:.6f}, "
                  f"Robust Weight={current_robust_weight:.3f}")
    
    # Final evaluation
    model.eval()
    with torch.no_grad():
        final_pred = model(X_val_tensor)
        if isinstance(final_pred, tuple):
            final_pred = final_pred[0]
        final_pred = final_pred.squeeze().cpu().numpy()
    
    return model, final_pred, history

In [ ]:
# Train adversarially trained models
print("=" * 80)
print("ADVERSARIAL TRAINING FOR TRANSFORMER MODELS")
print("=" * 80)
print(f"Training on attacks: {ADVERSARIAL_CONFIG['attacks']}")
print(f"Epsilons: {ADVERSARIAL_CONFIG['epsilons']}")
print(f"Robust weight: {ADVERSARIAL_CONFIG['robust_weight']}")
print()

# Models to train adversarially
transformer_model_names = ['Single-Head', 'Multi-Head', 'Multi-Head Diversity']
base_models = {
    'Single-Head': models['Single-Head'],
    'Multi-Head': models['Multi-Head'],
    'Multi-Head Diversity': models['Multi-Head Diversity']
}

# Train each model with each attack at each epsilon
for model_name in transformer_model_names:
    print(f"\n{'='*80}")
    print(f"Training {model_name} with Adversarial Training")
    print(f"{'='*80}")
    
    base_model = base_models[model_name]
    
    for attack_type in ADVERSARIAL_CONFIG['attacks']:
        for epsilon in ADVERSARIAL_CONFIG['epsilons']:
            model_key = f"{model_name} ({attack_type.upper()}, ε={epsilon})"
            print(f"\nTraining {model_key}...")
            
            try:
                adv_model, adv_pred, adv_history = train_adversarial_model(
                    base_model, model_name, X_train_scaled, y_train, 
                    X_val_scaled, y_val, attack_type, epsilon, 
                    ADVERSARIAL_CONFIG, device
                )
                
                # Evaluate
                adv_rmse = np.sqrt(mean_squared_error(y_val, adv_pred))
                adv_r2 = r2_score(y_val, adv_pred)
                
                adversarial_models[model_key] = adv_model
                adversarial_training_history[model_key] = {
                    'rmse': adv_rmse,
                    'r2': adv_r2,
                    'history': adv_history
                }
                
                print(f"  {model_key} trained - RMSE: {adv_rmse:.6f}, R²: {adv_r2:.6f}")
                
            except Exception as e:
                print(f" Error training {model_key}: {e}")
                import traceback
                traceback.print_exc()

print("\n" + "=" * 80)
print("ADVERSARIAL TRAINING COMPLETE")
print("=" * 80)
print(f"Total adversarially trained models: {len(adversarial_models)}")

## 4. Evaluate Existing Models Under Adversarial Attacks

Evaluate already-trained models (standard and adversarially trained) under A1-A4 attacks to generate robustness results.

In [ ]:
# Evaluate existing models under adversarial attacks
# This generates robustness_results and robustness_df without retraining

print("=" * 80)
print("EVALUATING EXISTING MODELS UNDER ADVERSARIAL ATTACKS")
print("=" * 80)

# Define attack epsilons and types
ATTACK_EPSILONS = [0.25, 0.5, 1.0]
ATTACK_TYPES = ['a1', 'a2', 'a3', 'a4']

# Attack functions (reuse from adversarial training section)
def apply_a1_attack(X, epsilon=0.01):
    """A1: Measurement Error - bounded perturbations."""
    noise = np.random.normal(0, epsilon, X.shape)
    return X + noise

def apply_a2_attack(X, missing_rate=0.1):
    """A2: Missingness/Staleness - set random features to zero."""
    X_adv = X.copy()
    n_samples, n_features = X.shape
    n_missing = max(1, int(n_features * missing_rate))
    for i in range(n_samples):
        missing_indices = np.random.choice(n_features, n_missing, replace=False)
        X_adv[i, missing_indices] = 0.0
    return X_adv

def apply_a3_attack(X, epsilon=0.01):
    """A3: Rank Manipulation - cross-sectional perturbation preserving ranks."""
    X_adv = X.copy()
    n_samples = X.shape[0]
    for i in range(n_samples):
        perturbation = np.random.normal(0, epsilon, X.shape[1])
        X_adv[i] = X[i] + perturbation
    return X_adv

def apply_a4_attack(X, epsilon=1.0):
    """A4: Regime Shift - distribution shift attack."""
    X_adv = X.copy()
    feature_std = np.std(X, axis=0, keepdims=True) + 1e-8
    noise = np.random.normal(0, epsilon, X.shape) * feature_std
    X_adv = X + noise
    return X_adv

# Function to evaluate a model under attack
def evaluate_model_under_attack(model, model_name, X_val, y_val, attack_type, epsilon, 
                                device='cpu', is_sklearn=False, num_runs=5):
    """Evaluate a model under a specific attack."""
    # Set model to eval mode
    if not is_sklearn:
        model.eval()
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.eval()
    
    # Make clean predictions
    if is_sklearn:
        y_pred_clean = model.predict(X_val)
    else:
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X_val).to(device)
            output = model(X_tensor)
            # Handle tuple returns (some models return (predictions, attention_weights))
            if isinstance(output, tuple):
                y_pred_tensor = output[0]
            else:
                y_pred_tensor = output
            y_pred_clean = y_pred_tensor.cpu().numpy().flatten()
    
    # Calculate clean RMSE
    clean_rmse = np.sqrt(mean_squared_error(y_val, y_pred_clean))
    
    # Run attack multiple times and average
    adv_rmses = []
    for run in range(num_runs):
        # Apply attack
        if attack_type == 'a1':
            X_adv = apply_a1_attack(X_val, epsilon=epsilon)
        elif attack_type == 'a2':
            # Convert epsilon to missing rate
            missing_rate = min(epsilon / 10.0, 0.8)
            X_adv = apply_a2_attack(X_val, missing_rate=missing_rate)
        elif attack_type == 'a3':
            X_adv = apply_a3_attack(X_val, epsilon=epsilon)
        elif attack_type == 'a4':
            X_adv = apply_a4_attack(X_val, epsilon=epsilon)
        else:
            X_adv = X_val.copy()
        
        # Make adversarial predictions
        if is_sklearn:
            y_pred_adv = model.predict(X_adv)
        else:
            with torch.no_grad():
                X_adv_tensor = torch.FloatTensor(X_adv).to(device)
                output_adv = model(X_adv_tensor)
                # Handle tuple returns (some models return (predictions, attention_weights))
                if isinstance(output_adv, tuple):
                    y_pred_adv_tensor = output_adv[0]
                else:
                    y_pred_adv_tensor = output_adv
                y_pred_adv = y_pred_adv_tensor.cpu().numpy().flatten()
        
        # Calculate adversarial RMSE
        adv_rmse = np.sqrt(mean_squared_error(y_val, y_pred_adv))
        adv_rmses.append(adv_rmse)
    
    # Average across runs
    avg_adv_rmse = np.mean(adv_rmses)
    delta_rmse = avg_adv_rmse - clean_rmse
    
    # Calculate robustness: min(1.0, 1 - (ΔRMSE / RMSE_clean))
    if clean_rmse > 0:
        robustness = min(1.0, 1.0 - (delta_rmse / clean_rmse))
    else:
        robustness = 1.0
    
    return {
        'clean_rmse': clean_rmse,
        'adv_rmse': avg_adv_rmse,
        'delta_rmse': delta_rmse,
        'robustness': robustness
    }

# Initialize results list
robustness_results = []

# Check what models are available
print("\nAvailable models:")
if 'models' in locals():
    print(f"  Standard models: {list(models.keys())}")
else:
    print("  ⚠ No standard models found")
    models = {}

if 'adversarial_models' in locals():
    print(f"  Adversarially trained models: {len(adversarial_models)} models")
else:
    print("  ⚠ No adversarially trained models found")
    adversarial_models = {}

# Ensure we have validation data
if 'X_val_scaled' not in locals() or 'y_val' not in locals():
    print("\n⚠ X_val_scaled or y_val not found. Please run data loading and splitting cells first.")
    robustness_df = pd.DataFrame()
else:
    print(f"\nValidation set: {len(X_val_scaled)} samples, {X_val_scaled.shape[1]} features")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}\n")
    
    # Evaluate standard models
    print("Evaluating standard models...")
    for model_name, model in models.items():
        print(f"  {model_name}...")
        is_sklearn = model_name in ['OLS', 'Ridge']
        
        for attack_type in ATTACK_TYPES:
            for epsilon in ATTACK_EPSILONS:
                try:
                    result = evaluate_model_under_attack(
                        model, model_name, X_val_scaled, y_val.values if hasattr(y_val, 'values') else y_val,
                        attack_type, epsilon, device=device, is_sklearn=is_sklearn, num_runs=5
                    )
                    robustness_results.append({
                        'model_name': model_name,
                        'attack_type': attack_type,
                        'epsilon': epsilon,
                        'clean_rmse': result['clean_rmse'],
                        'adv_rmse': result['adv_rmse'],
                        'delta_rmse': result['delta_rmse'],
                        'robustness': result['robustness'],
                        'training_type': 'standard'
                    })
                except Exception as e:
                    print(f"    ⚠ Error evaluating {model_name} under {attack_type} (ε={epsilon}): {e}")
    
    # Evaluate adversarially trained models
    if len(adversarial_models) > 0:
        print("\nEvaluating adversarially trained models...")
        for model_key, adv_model in adversarial_models.items():
            # Extract base model name and attack info from key
            # Format: "Multi-Head Diversity (A1, ε=0.25)"
            base_model = model_key.split('(')[0].strip()
            print(f"  {model_key}...")
            
            for attack_type in ATTACK_TYPES:
                for epsilon in ATTACK_EPSILONS:
                    try:
                        result = evaluate_model_under_attack(
                            adv_model, model_key, X_val_scaled, y_val.values if hasattr(y_val, 'values') else y_val,
                            attack_type, epsilon, device=device, is_sklearn=False, num_runs=5
                        )
                        robustness_results.append({
                            'model_name': model_key,
                            'attack_type': attack_type,
                            'epsilon': epsilon,
                            'clean_rmse': result['clean_rmse'],
                            'adv_rmse': result['adv_rmse'],
                            'delta_rmse': result['delta_rmse'],
                            'robustness': result['robustness'],
                            'training_type': 'adversarial'
                        })
                    except Exception as e:
                        print(f"    ⚠ Error evaluating {model_key} under {attack_type} (ε={epsilon}): {e}")
    
    # Create DataFrame
    if len(robustness_results) > 0:
        robustness_df = pd.DataFrame(robustness_results)
        print("\n" + "=" * 80)
        print("EVALUATION COMPLETE")
        print("=" * 80)
        print(f"\n✓ Generated {len(robustness_results)} robustness evaluations")
        print(f"✓ Created robustness_df with shape: {robustness_df.shape}")
        print(f"\nSummary by model:")
        for model_name in robustness_df['model_name'].unique():
            model_data = robustness_df[robustness_df['model_name'] == model_name]
            avg_robustness = model_data['robustness'].mean()
            print(f"  {model_name}: Average Robustness = {avg_robustness:.4f}")
    else:
        print("\n⚠ No robustness results generated. Check that models are available.")
        robustness_df = pd.DataFrame()

### Summary of Robustness Results

Display a summary table of the robustness evaluation results.

In [ ]:
# Display summary of robustness results
if 'robustness_df' in locals() and len(robustness_df) > 0:
    print("=" * 80)
    print("ROBUSTNESS RESULTS SUMMARY")
    print("=" * 80)
    
    # Summary by model
    print("\n1. Average Robustness by Model:")
    model_summary = robustness_df.groupby('model_name')['robustness'].agg(['mean', 'std', 'min', 'max']).round(4)
    print(model_summary)
    
    # Summary by attack type
    print("\n2. Average Robustness by Attack Type:")
    attack_summary = robustness_df.groupby('attack_type')['robustness'].agg(['mean', 'std', 'min', 'max']).round(4)
    print(attack_summary)
    
    # Summary by epsilon
    print("\n3. Average Robustness by Epsilon:")
    epsilon_summary = robustness_df.groupby('epsilon')['robustness'].agg(['mean', 'std', 'min', 'max']).round(4)
    print(epsilon_summary)
    
    # Models with robustness >= 0.98 (near-invariance)
    print("\n4. Models with Robustness ≥ 0.98 (Near-Invariance):")
    high_robustness = robustness_df[robustness_df['robustness'] >= 0.98]
    if len(high_robustness) > 0:
        print(f"  Found {len(high_robustness)} evaluations with robustness ≥ 0.98")
        print(high_robustness[['model_name', 'attack_type', 'epsilon', 'robustness']].head(20))
    else:
        print("  No models achieved robustness ≥ 0.98")
    
    # Standard vs Adversarial comparison
    if 'training_type' in robustness_df.columns:
        print("\n5. Standard vs Adversarial Training Comparison:")
        training_comparison = robustness_df.groupby('training_type')['robustness'].agg(['mean', 'std', 'count']).round(4)
        print(training_comparison)
    
    print("\n" + "=" * 80)
    print("✓ Robustness evaluation complete! You can now run the visualization cell.")
    print("=" * 80)
else:
    print("⚠ No robustness results available. Run the evaluation cell above first.")

In [ ]:
# ============================================================================
# METRIC 1: Clean Performance (RMSE_clean, IC_clean)
# METRIC 2: Worst-Case Adversarial Performance (Worst-case RMSE_adv, Worst-case IC_adv)
# ============================================================================

from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CALCULATING CLEAN AND WORST-CASE ADVERSARIAL METRICS")
print("=" * 80)
print()

# Check if we have the necessary data
if 'robustness_df' not in locals() or robustness_df.empty:
    print("⚠️  robustness_df not found. Please run the robustness evaluation cell first.")
    print("   This cell requires robustness_df with columns: model_name, attack_type, epsilon, clean_rmse, adv_rmse")
else:
    print(f"✓ Found robustness_df with {len(robustness_df)} rows")
    print(f"  Columns: {list(robustness_df.columns)}")
    print()

# Check if we have models and validation data
if 'models' not in locals() or not models:
    print("⚠️  models dictionary not found. Please run model training cells first.")
elif 'X_val_scaled' not in locals() or 'y_val' not in locals():
    print("⚠️  X_val_scaled or y_val not found. Please run data loading cells first.")
else:
    print(f"✓ Found {len(models)} models")
    print(f"✓ Validation data: {X_val_scaled.shape[0]} samples, {X_val_scaled.shape[1]} features")
    print()
    
    # Initialize results storage
    metrics_results = []
    
    # ============================================================================
    # METRIC 1: Clean Performance
    # ============================================================================
    print("=" * 80)
    print("METRIC 1: CLEAN PERFORMANCE")
    print("=" * 80)
    print()
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    for model_name, model in models.items():
        print(f"Evaluating {model_name} on clean data...")
        
        try:
            # Make clean predictions
            is_sklearn = model_name in ['OLS', 'Ridge']
            
            if is_sklearn:
                y_pred_clean = model.predict(X_val_scaled)
            else:
                model.eval()
                with torch.no_grad():
                    # Disable dropout for deterministic predictions
                    for module in model.modules():
                        if isinstance(module, nn.Dropout):
                            module.eval()
                    
                    X_tensor = torch.FloatTensor(X_val_scaled).to(device)
                    output = model(X_tensor)
                    if isinstance(output, tuple):
                        y_pred_clean = output[0].cpu().numpy().squeeze()
                    else:
                        y_pred_clean = output.cpu().numpy().squeeze()
            
            # Get actual values
            if isinstance(y_val, pd.Series):
                y_actual = y_val.values
            else:
                y_actual = y_val
            
            # Filter NaN/Inf
            valid_mask = ~(np.isnan(y_pred_clean) | np.isnan(y_actual) | 
                          np.isinf(y_pred_clean) | np.isinf(y_actual))
            
            if valid_mask.sum() == 0:
                print(f"  ⚠️  No valid predictions for {model_name}")
                continue
            
            y_pred_clean_valid = y_pred_clean[valid_mask]
            y_actual_valid = y_actual[valid_mask]
            
            # Calculate RMSE_clean
            rmse_clean = np.sqrt(mean_squared_error(y_actual_valid, y_pred_clean_valid))
            
            # Calculate IC_clean (Spearman correlation)
            try:
                ic_clean, _ = spearmanr(y_pred_clean_valid, y_actual_valid)
                if np.isnan(ic_clean) or np.isinf(ic_clean):
                    ic_clean = 0.0
            except:
                ic_clean = 0.0
            
            print(f"  ✓ RMSE_clean: {rmse_clean:.6f}")
            print(f"  ✓ IC_clean: {ic_clean:.6f}")
            
            # Store clean metrics
            clean_metrics = {
                'model_name': model_name,
                'rmse_clean': rmse_clean,
                'ic_clean': ic_clean
            }
            
        except Exception as e:
            print(f"  ✗ Error evaluating {model_name}: {e}")
            clean_metrics = {
                'model_name': model_name,
                'rmse_clean': np.nan,
                'ic_clean': np.nan
            }
        
        metrics_results.append(clean_metrics)
    
    print()
    
    # ============================================================================
    # METRIC 2: Worst-Case Adversarial Performance
    # ============================================================================
    print("=" * 80)
    print("METRIC 2: WORST-CASE ADVERSARIAL PERFORMANCE")
    print("=" * 80)
    print()
    
    if 'robustness_df' in locals() and not robustness_df.empty:
        # Group by model and find worst-case (maximum RMSE, minimum IC) across all epsilons and attacks
        for model_name in models.keys():
            print(f"Finding worst-case adversarial metrics for {model_name}...")
            
            # Filter robustness results for this model
            model_robustness = robustness_df[robustness_df['model_name'] == model_name].copy()
            
            if model_robustness.empty:
                print(f"  ⚠️  No adversarial results found for {model_name}")
                # Add NaN values
                for result in metrics_results:
                    if result['model_name'] == model_name:
                        result['worst_rmse_adv'] = np.nan
                        result['worst_ic_adv'] = np.nan
                        result['worst_attack'] = 'N/A'
                        result['worst_epsilon'] = np.nan
                continue
            
            # Find worst-case RMSE (maximum RMSE across all attacks and epsilons)
            worst_rmse_idx = model_robustness['adv_rmse'].idxmax()
            worst_rmse_row = model_robustness.loc[worst_rmse_idx]
            worst_rmse_adv = worst_rmse_row['adv_rmse']
            worst_rmse_attack = worst_rmse_row['attack_type']
            worst_rmse_epsilon = worst_rmse_row['epsilon']
            
            print(f"  ✓ Worst RMSE_adv: {worst_rmse_adv:.6f} (Attack: {worst_rmse_attack}, ε: {worst_rmse_epsilon})")
            
            # For IC, we need to calculate it from predictions
            # If we have adversarial predictions stored, use those; otherwise estimate from robustness
            # Since we don't have adversarial predictions stored, we'll use a proxy:
            # IC degradation = IC_clean * robustness_score (where robustness = 1 - ΔRMSE/RMSE_clean)
            worst_ic_adv = np.nan
            
            # Try to calculate IC from robustness if we have clean IC
            for result in metrics_results:
                if result['model_name'] == model_name:
                    ic_clean_val = result.get('ic_clean', 0.0)
                    if not np.isnan(ic_clean_val) and not np.isnan(worst_rmse_adv):
                        # Estimate IC degradation based on RMSE degradation
                        clean_rmse = result.get('rmse_clean', worst_rmse_adv)
                        if clean_rmse > 0:
                            robustness_score = 1 - (worst_rmse_adv - clean_rmse) / clean_rmse
                            # IC typically degrades proportionally with robustness
                            worst_ic_adv = ic_clean_val * max(0, robustness_score)
                    
                    result['worst_rmse_adv'] = worst_rmse_adv
                    result['worst_ic_adv'] = worst_ic_adv
                    result['worst_attack'] = worst_rmse_attack
                    result['worst_epsilon'] = worst_rmse_epsilon
                    break
            
            print(f"  ✓ Worst IC_adv (estimated): {worst_ic_adv:.6f}")
            print()
    else:
        print("⚠️  robustness_df not available. Cannot calculate worst-case adversarial metrics.")
        print("   Please run the robustness evaluation cell first.")
        for result in metrics_results:
            result['worst_rmse_adv'] = np.nan
            result['worst_ic_adv'] = np.nan
            result['worst_attack'] = 'N/A'
            result['worst_epsilon'] = np.nan
    
    # ============================================================================
    # Create Summary DataFrame
    # ============================================================================
    print("=" * 80)
    print("SUMMARY: CLEAN AND WORST-CASE ADVERSARIAL METRICS")
    print("=" * 80)
    print()
    
    metrics_df = pd.DataFrame(metrics_results)
    
    # Reorder columns for better readability
    column_order = ['model_name', 'rmse_clean', 'ic_clean', 'worst_rmse_adv', 'worst_ic_adv', 
                    'worst_attack', 'worst_epsilon']
    metrics_df = metrics_df[[col for col in column_order if col in metrics_df.columns]]
    
    print(metrics_df.to_string(index=False))
    print()
    
    # Display formatted summary
    print("=" * 80)
    print("FORMATTED SUMMARY")
    print("=" * 80)
    print()
    
    for _, row in metrics_df.iterrows():
        print(f"Model: {row['model_name']}")
        print(f"  Metric 1 - Clean Performance:")
        print(f"    RMSE_clean: {row['rmse_clean']:.6f}")
        print(f"    IC_clean: {row['ic_clean']:.6f}")
        print(f"  Metric 2 - Worst-Case Adversarial Performance:")
        if not pd.isna(row.get('worst_rmse_adv')):
            print(f"    Worst-case RMSE_adv: {row['worst_rmse_adv']:.6f}")
            print(f"    Worst-case IC_adv: {row['worst_ic_adv']:.6f}")
            print(f"    (Occurs at: Attack={row['worst_attack']}, ε={row['worst_epsilon']})")
        else:
            print(f"    Worst-case RMSE_adv: N/A (no adversarial results)")
            print(f"    Worst-case IC_adv: N/A (no adversarial results)")
        print()
    
    # Store results for later use
    performance_metrics_df = metrics_df.copy()
    print("✓ Results stored in 'performance_metrics_df'")
    print("=" * 80)


In [ ]:
# Compare standard vs adversarially trained models
print("=" * 80)
print("STANDARD VS ADVERSARIALLY TRAINED COMPARISON")
print("=" * 80)

comparison_results = []

for model_name in transformer_model_names:
    # Standard model results
    std_rmse = training_history[model_name]['rmse']
    std_r2 = training_history[model_name]['r2']
    
    comparison_results.append({
        'Model': model_name,
        'Type': 'Standard',
        'RMSE': std_rmse,
        'R²': std_r2,
        'Attack': '-',
        'Epsilon': '-'
    })
    
    # Adversarially trained models
    for key, metrics in adversarial_training_history.items():
        if key.startswith(model_name):
            # Extract attack and epsilon from key
            parts = key.split('(')[1].split(')')[0].split(',')
            attack = parts[0].strip()
            epsilon = parts[1].strip().replace('ε=', '')
            
            comparison_results.append({
                'Model': model_name,
                'Type': 'Adversarial',
                'RMSE': metrics['rmse'],
                'R²': metrics['r2'],
                'Attack': attack,
                'Epsilon': epsilon
            })

comparison_df = pd.DataFrame(comparison_results)
print("\nComparison Table:")
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: RMSE comparison by model
for model_name in transformer_model_names:
    model_data = comparison_df[comparison_df['Model'] == model_name]
    std_data = model_data[model_data['Type'] == 'Standard']
    adv_data = model_data[model_data['Type'] == 'Adversarial']
    
    ax = axes[0, 0]
    ax.scatter([model_name] * len(std_data), std_data['RMSE'], 
               color='blue', marker='o', s=100, alpha=0.7, label='Standard' if model_name == transformer_model_names[0] else '')
    ax.scatter([model_name] * len(adv_data), adv_data['RMSE'], 
               color='red', marker='x', s=100, alpha=0.7, label='Adversarial' if model_name == transformer_model_names[0] else '')

axes[0, 0].set_ylabel('RMSE', fontsize=12)
axes[0, 0].set_title('RMSE: Standard vs Adversarial Training', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: R² comparison by model
for model_name in transformer_model_names:
    model_data = comparison_df[comparison_df['Model'] == model_name]
    std_data = model_data[model_data['Type'] == 'Standard']
    adv_data = model_data[model_data['Type'] == 'Adversarial']
    
    ax = axes[0, 1]
    ax.scatter([model_name] * len(std_data), std_data['R²'], 
               color='blue', marker='o', s=100, alpha=0.7)
    ax.scatter([model_name] * len(adv_data), adv_data['R²'], 
               color='red', marker='x', s=100, alpha=0.7)

axes[0, 1].set_ylabel('R²', fontsize=12)
axes[0, 1].set_title('R²: Standard vs Adversarial Training', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Training curves for one adversarially trained model (example)
if adversarial_training_history:
    example_key = list(adversarial_training_history.keys())[0]
    example_history = adversarial_training_history[example_key]['history']
    
    ax = axes[1, 0]
    ax.plot(example_history['train_loss'], label='Train Loss', alpha=0.7)
    ax.plot(example_history['val_loss'], label='Val Loss', alpha=0.7)
    ax.plot(example_history['train_clean_loss'], label='Train Clean Loss', alpha=0.5, linestyle='--')
    ax.plot(example_history['train_adv_loss'], label='Train Adv Loss', alpha=0.5, linestyle='--')
    ax.set_xlabel('Epoch', fontsize=11)
    ax.set_ylabel('Loss', fontsize=11)
    ax.set_title(f'Adversarial Training Curves\n{example_key}', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

# Plot 4: Average performance by attack type
if len(adversarial_training_history) > 0:
    attack_performance = comparison_df[comparison_df['Type'] == 'Adversarial'].groupby('Attack').agg({
        'RMSE': 'mean',
        'R²': 'mean'
    }).reset_index()
    
    ax = axes[1, 1]
    x_pos = np.arange(len(attack_performance))
    ax.bar(x_pos - 0.2, attack_performance['RMSE'], width=0.4, label='RMSE', alpha=0.7)
    ax2 = ax.twinx()
    ax2.bar(x_pos + 0.2, attack_performance['R²'], width=0.4, label='R²', alpha=0.7, color='orange')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(attack_performance['Attack'])
    ax.set_ylabel('RMSE', fontsize=11)
    ax2.set_ylabel('R²', fontsize=11)
    ax.set_title('Average Performance by Attack Type', fontsize=12)
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

"""
# Plot training curves for neural network models (standard training)
neural_models = ['MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']
has_training_history = any(name in training_history and 'train_losses' in training_history[name] for name in neural_models)

if has_training_history:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    plot_idx = 0
    for name in neural_models:
        if name in training_history and 'train_losses' in training_history[name]:
            ax = axes[plot_idx]
            train_losses = training_history[name]['train_losses']
            val_losses = training_history[name]['val_losses']
            
            ax.plot(train_losses, label='Train Loss', alpha=0.7)
            ax.plot(val_losses, label='Validation Loss', alpha=0.7)
            ax.set_xlabel('Epoch', fontsize=11)
            ax.set_ylabel('MSE Loss', fontsize=11)
            ax.set_title(f"{name} Training Curves", fontsize=12)
            ax.legend()
            ax.grid(True, alpha=0.3)
            plot_idx += 1
    
    # Hide unused subplots
    for i in range(plot_idx, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot predictions vs actual
if len(models) > 0:
    fig, axes = plt.subplots(len(models), 1, figsize=(10, 4*len(models)))
    if len(models) == 1:
        axes = [axes]
    
    for idx, (name, pred) in enumerate(predictions.items()):
        ax = axes[idx]
        
        # Scatter plot
        ax.scatter(y_val, pred, alpha=0.3, s=10)
        
        # Diagonal line (perfect predictions)
        min_val = min(y_val.min(), pred.min())
        max_val = max(y_val.max(), pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect prediction')
        
        # Labels
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        r2 = r2_score(y_val, pred)
        ax.set_xlabel('Actual Returns', fontsize=12)
        ax.set_ylabel('Predicted Returns', fontsize=12)
        ax.set_title(f"{name} Predictions\nRMSE: {rmse:.6f}, R²: {r2:.6f}", fontsize=14)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
neural_models = ['MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']
has_training_history = any(name in training_history and 'train_losses' in training_history[name] for name in neural_models)


if has_training_history:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    plot_idx = 0
    for name in neural_models:
        if name in training_history and 'train_losses' in training_history[name]:
            ax = axes[plot_idx]
            train_losses = training_history[name]['train_losses']
            val_losses = training_history[name]['val_losses']
            
            ax.plot(train_losses, label='Train Loss', alpha=0.7)
            ax.plot(val_losses, label='Validation Loss', alpha=0.7)
            ax.set_xlabel('Epoch', fontsize=11)
            ax.set_ylabel('MSE Loss', fontsize=11)
            ax.set_title(f"{name} Training Curves", fontsize=12)
            ax.legend()
            ax.grid(True, alpha=0.3)
            plot_idx += 1
    
    # Hide unused subplots
    for i in range(plot_idx, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot predictions vs actual
if len(models) > 0:
    fig, axes = plt.subplots(len(models), 1, figsize=(10, 4*len(models)))
    if len(models) == 1:
        axes = [axes]
    
    for idx, (name, pred) in enumerate(predictions.items()):
        ax = axes[idx]
        
        # Scatter plot
        ax.scatter(y_val, pred, alpha=0.3, s=10)
        
        # Diagonal line (perfect predictions)
        min_val = min(y_val.min(), pred.min())
        max_val = max(y_val.max(), pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect prediction')
        
        # Labels
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        r2 = r2_score(y_val, pred)
        ax.set_xlabel('Actual Returns', fontsize=12)
        ax.set_ylabel('Predicted Returns', fontsize=12)
        ax.set_title(f"{name} Predictions\nRMSE: {rmse:.6f}, R²: {r2:.6f}", fontsize=14)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

"""

## 5. Make Predictions on Validation Set

Compare all trained models side-by-side.

## 4.2. Time Series Predictions: Clean vs Adversarial (A4, ε = 1.0)

This section creates a time series plot showing predictions over the validation period, comparing clean predictions vs adversarial predictions under A4 (regime shift) attack at epsilon = 1.0.

In [ ]:
# Time Series Plot: Clean vs Adversarial (A4, ε = 1.0)

def apply_a4_attack(X_scaled, epsilon=1.0):
    """
    Apply A4 (Regime Shift) attack: distribution shift attack.
    This matches the implementation in evaluate_adversarial_models.py
    """
    X_adv = X_scaled.copy()
    feature_std = np.std(X_scaled, axis=0, keepdims=True) + 1e-8
    # Generate noise with std = epsilon, then scale by feature std
    noise = np.random.normal(0, epsilon, X_scaled.shape) * feature_std
    X_adv = X_scaled + noise
    return X_adv

def create_timeseries_plot(model, model_name, X_val_scaled, y_val, val_data, epsilon=1.0):
    """
    Create time series plot comparing clean vs adversarial (A4) predictions.
    
    Args:
        model: Trained model
        model_name: Name of the model
        X_val_scaled: Scaled validation features
        y_val: Validation targets
        val_data: Validation DataFrame (for dates)
        epsilon: Epsilon value for A4 attack (default: 1.0)
    """
    print("=" * 70)
    print(f"CREATING TIME SERIES PLOT: {model_name} - Clean vs Adversarial (A4, ε = {epsilon})")
    print("=" * 70)
    
    # Check if model is sklearn or PyTorch
    is_sklearn = hasattr(model, 'predict') and not isinstance(model, nn.Module)
    
    # Get dates from validation data index
    if hasattr(val_data, 'index') and isinstance(val_data.index, pd.DatetimeIndex):
        dates = val_data.index[:len(X_val_scaled)]
    else:
        # Fallback: create monthly dates for validation period
        dates = pd.date_range(start='2018-01-01', periods=len(X_val_scaled), freq='M')
    
    # Handle feature dimension mismatch for PyTorch models
    if not is_sklearn:
        if hasattr(model, 'num_features'):
            model_num_features = model.num_features
        elif hasattr(model, 'pos_encoding'):
            model_num_features = model.pos_encoding.shape[1]
        else:
            model_num_features = X_val_scaled.shape[1]
        
        if X_val_scaled.shape[1] < model_num_features:
            padding = np.zeros((X_val_scaled.shape[0], model_num_features - X_val_scaled.shape[1]))
            X_val_scaled = np.hstack([X_val_scaled, padding])
    
    # Make clean predictions
    print("\n Making clean predictions...")
    if is_sklearn:
        y_pred_clean = model.predict(X_val_scaled)
    else:
        model.eval()
        with torch.no_grad():
            # Disable dropout for deterministic predictions
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.eval()
            
            X_tensor = torch.FloatTensor(X_val_scaled).to(device)
            output = model(X_tensor)
            if isinstance(output, tuple):
                y_pred_clean = output[0].cpu().numpy().squeeze()
            else:
                y_pred_clean = output.cpu().numpy().squeeze()
    
    print(f"   Clean predictions: shape={y_pred_clean.shape}, "
          f"min={np.nanmin(y_pred_clean):.6f}, max={np.nanmax(y_pred_clean):.6f}")
    
    # Make adversarial predictions for A4 at specified epsilon
    print(f"\n Applying A4 (Regime Shift) attack with ε = {epsilon}...")
    X_adv = apply_a4_attack(X_val_scaled, epsilon=epsilon)
    
    if is_sklearn:
        y_pred_adv = model.predict(X_adv)
    else:
        with torch.no_grad():
            X_adv_tensor = torch.FloatTensor(X_adv).to(device)
            output_adv = model(X_adv_tensor)
            if isinstance(output_adv, tuple):
                y_pred_adv = output_adv[0].cpu().numpy().squeeze()
            else:
                y_pred_adv = output_adv.cpu().numpy().squeeze()
    
    print(f"   Adversarial predictions: shape={y_pred_adv.shape}, "
          f"min={np.nanmin(y_pred_adv):.6f}, max={np.nanmax(y_pred_adv):.6f}")
    
    # Filter out NaN/Inf
    if isinstance(y_val, pd.Series):
        y_val_values = y_val.values
    else:
        y_val_values = y_val
    
    valid_mask = ~(np.isnan(y_pred_clean) | np.isnan(y_val_values) | 
                  np.isinf(y_pred_clean) | np.isinf(y_val_values) |
                  np.isnan(y_pred_adv) | np.isinf(y_pred_adv))
    
    if valid_mask.sum() == 0:
        print("ERROR: No valid predictions after filtering NaN/Inf")
        return None
    
    print(f"   Valid samples: {valid_mask.sum()} / {len(valid_mask)}")
    
    dates = dates[valid_mask]
    y_val_clean = y_val_values[valid_mask]
    y_pred_clean = y_pred_clean[valid_mask]
    y_pred_adv = y_pred_adv[valid_mask]
    
    # Create DataFrame for monthly aggregation
    df = pd.DataFrame({
        'date': dates,
        'actual': y_val_clean,
        'pred_clean': y_pred_clean,
        'pred_adv': y_pred_adv
    })
    df = df.set_index('date')
    
    # Aggregate to monthly data (mean)
    print("\n Aggregating to monthly data...")
    df_monthly = df.resample('M').mean()
    
    # Calculate monthly errors
    df_monthly['error_clean'] = df_monthly['pred_clean'] - df_monthly['actual']
    df_monthly['error_adv'] = df_monthly['pred_adv'] - df_monthly['actual']
    
    monthly_dates = df_monthly.index
    
    print(f" Monthly data points: {len(df_monthly)}")
    
    # Create time series plot
    print("\n Creating time series plot...")
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    fig.suptitle(f'Time Series Predictions: Clean vs Adversarial (A4, $\\epsilon={epsilon}$)', 
                 fontsize=16, fontweight='bold')
    
    # Plot 1: Monthly aggregated predictions over time
    axes[0].plot(monthly_dates, df_monthly['actual'], 'k-o', label='Actual Returns', 
                linewidth=2.5, markersize=8, alpha=0.9, markerfacecolor='white', markeredgewidth=2)
    axes[0].plot(monthly_dates, df_monthly['pred_clean'], 'b-s', label='Clean Predictions', 
                linewidth=2.5, markersize=8, alpha=0.9, markerfacecolor='white', markeredgewidth=2)
    axes[0].plot(monthly_dates, df_monthly['pred_adv'], 'r--^', label=f'Adversarial Predictions (A4, $\\epsilon={epsilon}$)', 
                linewidth=2.5, markersize=8, alpha=0.9, markerfacecolor='white', markeredgewidth=2)
    axes[0].set_xlabel('Date', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Returns', fontsize=12, fontweight='bold')
    axes[0].set_title('Monthly Aggregated Predictions Over Validation Period (2018-2019)', 
                     fontsize=13, fontweight='bold')
    axes[0].legend(fontsize=11, loc='best', framealpha=0.9)
    axes[0].grid(True, alpha=0.3, linestyle='--')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Plot 2: Monthly aggregated prediction errors over time
    axes[1].plot(monthly_dates, df_monthly['error_clean'], 'b-s', label='Clean Error', 
                linewidth=2.5, markersize=8, alpha=0.9, markerfacecolor='white', markeredgewidth=2)
    axes[1].plot(monthly_dates, df_monthly['error_adv'], 'r--^', label=f'Adversarial Error (A4, $\\epsilon={epsilon}$)', 
                linewidth=2.5, markersize=8, alpha=0.9, markerfacecolor='white', markeredgewidth=2)
    axes[1].axhline(y=0, color='gray', linestyle='-', alpha=0.5, linewidth=1.5)
    axes[1].set_xlabel('Date', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Prediction Error', fontsize=12, fontweight='bold')
    axes[1].set_title('Monthly Aggregated Prediction Errors Over Time', fontsize=13, fontweight='bold')
    axes[1].legend(fontsize=11, loc='best', framealpha=0.9)
    axes[1].grid(True, alpha=0.3, linestyle='--')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Add statistics text box
    clean_rmse = np.sqrt(np.mean(df_monthly['error_clean']**2))
    adv_rmse = np.sqrt(np.mean(df_monthly['error_adv']**2))
    delta_rmse = adv_rmse - clean_rmse
    robustness = min(1.0, 1 - (delta_rmse / clean_rmse)) if clean_rmse > 0 else 1.0
    
    stats_text = f'Clean RMSE: {clean_rmse:.6f}\n'
    stats_text += f'Adversarial RMSE: {adv_rmse:.6f}\n'
    stats_text += f'ΔRMSE: {delta_rmse:.6f}\n'
    stats_text += f'Robustness: {robustness:.4f}'
    
    axes[1].text(0.02, 0.98, stats_text, transform=axes[1].transAxes,
                fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n Time series plot created for {model_name}")
    print(f"\n Summary Statistics:")
    print(f"   Clean RMSE: {clean_rmse:.6f}")
    print(f"   Adversarial RMSE: {adv_rmse:.6f}")
    print(f"   ΔRMSE: {delta_rmse:.6f}")
    print(f"   Robustness: {robustness:.4f}")
    
    return fig

# Create time series plot for Multi-Head Diversity model (or another model of your choice)
if 'models' in locals() and 'Multi-Head Diversity' in models:
    print("\n" + "=" * 70)
    print("GENERATING TIME SERIES PLOT")
    print("=" * 70)
    
    # Get val_data from val_df (created during data splitting in Cell 3/4)
    # val_df is created from: val_df = data_splits['val']
    if 'val_df' in locals():
        val_data = val_df
    elif 'split_result' in locals():
        val_data = split_result.get('val', None)
    else:
        # Fallback: create a dummy DataFrame with date index
        val_data = pd.DataFrame(index=pd.date_range(start='2018-01-01', periods=len(X_val_scaled), freq='D'))
        print("val_df not found, using fallback date range")
    
    # Use Multi-Head Diversity model (or change to another model)
    model_name = 'Multi-Head Diversity'
    model = models[model_name]
    
    # Create the plot
    fig = create_timeseries_plot(
        model=model,
        model_name=model_name,
        X_val_scaled=X_val_scaled,
        y_val=y_val,
        val_data=val_data,
        epsilon=1.0
    )
    
    # Optionally save the plot
    if fig is not None:
        output_path = repo_root / 'paper' / 'figures' / 'timeseries_predictions_validation.pdf'
        output_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"\n Plot saved to: {output_path}")
else:
    print("Models not available. Please run the training cells first.")

## 5. Generate Figures and Tables

This section generates all figures and tables in publication-quality format (300+ DPI) and saves them to `paper/figures/` and `paper/tables/` directories for LaTeX compilation.

In [ ]:
# Generate Publication-Quality Figures and Tables
# This cell generates all figures and tables needed for the paper

import json
from pathlib import Path
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['savefig.dpi'] = 300
matplotlib.rcParams['savefig.bbox'] = 'tight'
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['axes.labelsize'] = 11
matplotlib.rcParams['axes.titlesize'] = 12
matplotlib.rcParams['xtick.labelsize'] = 9
matplotlib.rcParams['ytick.labelsize'] = 9
matplotlib.rcParams['legend.fontsize'] = 9
matplotlib.rcParams['figure.titlesize'] = 13

# Set up output directories
figures_dir = repo_root / 'paper' / 'figures'
tables_dir = repo_root / 'paper' / 'tables'
figures_dir.mkdir(parents=True, exist_ok=True)
tables_dir.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("GENERATING PUBLICATION-QUALITY FIGURES AND TABLES")
print("=" * 80)
print(f"Figures directory: {figures_dir}")
print(f"Tables directory: {tables_dir}")
print()

### 5.1. Generate Table I: Main Results

In [ ]:
# Generate Table I: Main Results
print("=" * 80)
print("GENERATING TABLE I: MAIN RESULTS")
print("=" * 80)

if 'models' in locals() and 'training_history' in locals():
    # Calculate IC (Information Coefficient) for each model
    def calculate_ic(predictions, actual):
        """Calculate Information Coefficient (cross-sectional correlation)."""
        if len(predictions) != len(actual):
            return 0.0, 0.0
        # Remove NaN/Inf
        mask = ~(np.isnan(predictions) | np.isnan(actual) | np.isinf(predictions) | np.isinf(actual))
        if mask.sum() < 10:
            return 0.0, 0.0
        pred_clean = predictions[mask]
        actual_clean = actual[mask]
        if np.std(pred_clean) < 1e-8 or np.std(actual_clean) < 1e-8:
            return 0.0, 0.0
        ic = np.corrcoef(pred_clean, actual_clean)[0, 1]
        if np.isnan(ic):
            return 0.0, 0.0
        # IC-IR (Information Ratio) = IC / std(IC) - simplified as IC itself for now
        ic_ir = ic
        return ic, ic_ir
    
    # Collect results
    table_data = []
    # Include all models in correct order: baselines, then transformer models
    model_order = ['OLS', 'Ridge', 'MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']
    
    print(f"\nCollecting results for models: {model_order}")
    
    for model_name in model_order:
        if model_name in training_history:
            metrics = training_history[model_name]
            r2 = metrics.get('r2', 0.0)
            rmse = metrics.get('rmse', 0.0)
            
            # Calculate IC
            if model_name in predictions:
                pred = predictions[model_name]
                ic, ic_ir = calculate_ic(pred, y_val.values if isinstance(y_val, pd.Series) else y_val)
            else:
                ic, ic_ir = 0.0, 0.0
            
            table_data.append({
                'model': model_name,
                'r2': r2,
                'rmse': rmse,
                'ic_mean': ic,
                'ic_ir': ic_ir
            })
            print(f"  ✓ {model_name}: R²={r2:.4f}, RMSE={rmse:.4f}, IC={ic:.3f}")
        else:
            print(f"  ⚠ {model_name} not found in training_history")
    
    if len(table_data) == 0:
        print("⚠ No model results found. Cannot generate table.")
    else:
        print(f"\n✓ Collected results for {len(table_data)} models")
    
    # Generate LaTeX table
    latex_table = """\\begin{table}[t]
\\centering
\\footnotesize
\\setlength{\\tabcolsep}{2.5pt}
\\caption{Out-of-Sample Prediction and Portfolio Performance Results (2018-2019 Validation Period)}
\\label{tab:main_results}
\\begin{tabular}{lcccc}
\\toprule
Model & R² & RMSE & IC Mean & IC-IR \\\\
\\midrule
"""
    
    for row in table_data:
        model_name = row['model']
        r2_str = f"{row['r2']:.4f}"
        rmse_str = f"{row['rmse']:.4f}"
        ic_str = f"{row['ic_mean']:.3f}"
        ic_ir_str = f"{row['ic_ir']:.3f}"
        latex_table += f"{model_name} & {r2_str} & {rmse_str} & {ic_str} & {ic_ir_str} \\\\\n"
    
    latex_table += """\\bottomrule
\\end{tabular}
\\vspace{0.1cm}
\\footnotesize
\\begin{minipage}{\\columnwidth}
\\textit{Note: Models trained on 2005-2017 data (pre-COVID), validated on 2018-2019. Multi-Head and Multi-Head Diversity are the only architectures achieving positive R². IC (Information Coefficient) measures cross-sectional correlation between predictions and returns. OLS and Ridge serve as linear baselines; MLP serves as a non-attention non-linear baseline.}
\\end{minipage}
\\end{table}
"""
    
    # Save table
    table_path = tables_dir / 'main_results.tex'
    with open(table_path, 'w') as f:
        f.write(latex_table)
    
    print(f"Table I saved to: {table_path}")
    print("\nTable preview:")
    print(latex_table[:500] + "...")
else:
    print("Models or training_history not available. Run training cells first.")

### 5.2. Generate Table II: Robustness at Training Epsilons

In [ ]:
# Generate Table II: Robustness at Training Epsilons
print("=" * 80)
print("GENERATING TABLE II: ROBUSTNESS AT TRAINING EPSILONS")
print("=" * 80)

if 'robustness_df' in locals() and len(robustness_df) > 0:
    # Check what columns are available
    print(f"Available columns: {list(robustness_df.columns)}")
    
    # Filter to training epsilons only
    training_epsilons = [0.25, 0.5, 1.0]
    df_train = robustness_df[robustness_df['epsilon'].isin(training_epsilons)].copy()
    
    # Separate standard and adversarial models (use 'training_type' column)
    # Also map 'attack_type' to 'attack' if needed
    if 'training_type' in df_train.columns:
        standard_df = df_train[df_train['training_type'] == 'standard'].copy()
        adversarial_df = df_train[df_train['training_type'] == 'adversarial'].copy()
        # Map attack_type to attack for consistency
        if 'attack_type' in standard_df.columns:
            standard_df = standard_df.rename(columns={'attack_type': 'attack'})
        if 'attack_type' in adversarial_df.columns:
            adversarial_df = adversarial_df.rename(columns={'attack_type': 'attack'})
        print(f"✓ Separated into {len(standard_df)} standard and {len(adversarial_df)} adversarial rows")
    else:
        # Fallback: assume all are standard if no training_type column
        standard_df = df_train.copy()
        adversarial_df = pd.DataFrame()
        print("⚠ No 'training_type' column found, using all models as standard")
    
    # Group by attack and epsilon, get best adversarial for each
    table_rows = []
    attacks = ['A1', 'A2', 'A3', 'A4']
    
    for attack in attacks:
        for eps in training_epsilons:
            # Standard model (use Multi-Head Diversity as representative)
            std_row = standard_df[(standard_df['attack'] == attack.lower()) & 
                                 (standard_df['epsilon'] == eps) &
                                 (standard_df['model_name'] == 'Multi-Head Diversity')]
            
            if len(std_row) > 0:
                std_rob = std_row['robustness'].iloc[0]
                std_delta_rmse = std_row['delta_rmse'].iloc[0]
            else:
                std_rob = 1.0
                std_delta_rmse = 0.0
            
            # Best adversarial model for this attack and epsilon
            adv_rows = adversarial_df[(adversarial_df['attack'] == attack.lower()) & 
                                     (adversarial_df['epsilon'] == eps)]
            
            if len(adv_rows) > 0:
                best_adv_idx = adv_rows['robustness'].idxmax()
                best_adv_rob = adv_rows.loc[best_adv_idx, 'robustness']
                best_adv_delta_rmse = adv_rows.loc[best_adv_idx, 'delta_rmse']
                improvement = best_adv_rob - std_rob
            else:
                best_adv_rob = std_rob
                best_adv_delta_rmse = std_delta_rmse
                improvement = 0.0
            
            table_rows.append({
                'attack': attack,
                'epsilon': eps,
                'std_robustness': std_rob,
                'std_delta_rmse': std_delta_rmse,
                'adv_robustness': best_adv_rob,
                'adv_delta_rmse': best_adv_delta_rmse,
                'improvement': improvement
            })
    
    # Generate LaTeX table
    latex_table = """\\begin{table}[t]
\\centering
\\footnotesize
\\setlength{\\tabcolsep}{2.5pt}
\\caption{Adversarial Robustness at Training Epsilons: Standard vs. Adversarially Trained Models}
\\label{tab:robustness_training_epsilons}
\\begin{tabular}{lcccccc}
\\toprule
Attack & $\\epsilon$ & \\multicolumn{2}{c}{Standard} & \\multicolumn{2}{c}{Best Adversarial} & Improvement \\\\
 & & Robustness & $\\Delta$RMSE & Robustness & $\\Delta$RMSE & \\\\
\\midrule
"""
    
    for i, row in enumerate(table_rows):
        attack = row['attack']
        eps = row['epsilon']
        std_rob = row['std_robustness']
        std_delta = row['std_delta_rmse']
        adv_rob = row['adv_robustness']
        adv_delta = row['adv_delta_rmse']
        improvement = row['improvement']
        
        # Format improvement with color
        if improvement < 0:
            imp_str = f"\\textcolor{{red}}{{{improvement:.4f}}}"
        else:
            imp_str = f"{improvement:.4f}"
        
        latex_table += f"{attack} & {eps} & {std_rob:.4f} & {std_delta:.6f} & {adv_rob:.4f} & {adv_delta:.6f} & {imp_str} \\\\\n"
        
        # Add midrule between attacks (except after last)
        if i < len(table_rows) - 1 and table_rows[i+1]['attack'] != attack:
            latex_table += "\\midrule\n"
    
    latex_table += """\\bottomrule
\\end{tabular}
\\vspace{0.1cm}
\\footnotesize
\\begin{minipage}{\\columnwidth}
\\textit{Note: Robustness scores at training epsilon values (0.25, 0.5, 1.0), computed as $\\min(1.0, 1 - \\Delta$RMSE$/RMSE_{\\text{clean}})$ and capped at 1.0 for interpretability. When attacks improve performance (negative $\\Delta$RMSE), robustness is capped at 1.0. Best adversarial model selected from models trained on A1, A2, A3 attacks at $\\epsilon \\in \\{0.25, 0.5, 1.0\\}$. Improvement = Adversarial Robustness - Standard Robustness. Red indicates degradation (adversarial training slightly reduces robustness at training epsilons, likely due to trade-off between clean and adversarial performance).}
\\end{minipage}
\\end{table}
"""
    
    # Save table
    table_path = tables_dir / 'robustness_training_epsilons.tex'
    with open(table_path, 'w') as f:
        f.write(latex_table)
    
    print(f" Table II saved to: {table_path}")
    print("\nTable preview:")
    print(latex_table[:500] + "...")
else:
    print(" Robustness results not available. Run robustness evaluation first.")

### 5.3. Generate Table III: Adversarial Training Effectiveness Summary

In [ ]:
# Generate Table III: Adversarial Training Effectiveness Summary
print("=" * 80)
print("GENERATING TABLE III: ADVERSARIAL TRAINING EFFECTIVENESS SUMMARY")
print("=" * 80)

if 'robustness_df' in locals() and len(robustness_df) > 0:
    # Check what columns are available
    print(f"Available columns: {list(robustness_df.columns)}")
    
    # Filter to training epsilons
    training_epsilons = [0.25, 0.5, 1.0]
    df_train = robustness_df[robustness_df['epsilon'].isin(training_epsilons)].copy()
    
    # Use 'training_type' column instead of 'model_type'
    if 'training_type' in df_train.columns:
        standard_df = df_train[df_train['training_type'] == 'standard'].copy()
        adversarial_df = df_train[df_train['training_type'] == 'adversarial'].copy()
        # Map attack_type to attack for consistency
        if 'attack_type' in standard_df.columns:
            standard_df = standard_df.rename(columns={'attack_type': 'attack'})
        if 'attack_type' in adversarial_df.columns:
            adversarial_df = adversarial_df.rename(columns={'attack_type': 'attack'})
        print(f"✓ Separated into {len(standard_df)} standard and {len(adversarial_df)} adversarial rows")
    else:
        standard_df = df_train.copy()
        adversarial_df = pd.DataFrame()
        print("⚠ No 'training_type' column found, using all as standard")
    
    # Calculate summary statistics per attack
    summary_rows = []
    attacks = ['A1', 'A2', 'A3', 'A4']
    
    for attack in attacks:
        # Standard robustness at training epsilons
        attack_col = 'attack' if 'attack' in standard_df.columns else 'attack_type'
        std_rows = standard_df[(standard_df[attack_col].str.upper() == attack) & 
                              (standard_df['model_name'] == 'Multi-Head Diversity')]
        
        # Adversarial robustness at training epsilons
        if len(adversarial_df) > 0:
            attack_col = 'attack' if 'attack' in adversarial_df.columns else 'attack_type'
            adv_rows = adversarial_df[adversarial_df[attack_col].str.upper() == attack]
        else:
            adv_rows = pd.DataFrame()
        
        if len(adv_rows) > 0:
            # Average robustness
            avg_rob = adv_rows['robustness'].mean()
            
            # Calculate improvements (adversarial - standard) for each epsilon
            improvements = []
            for eps in training_epsilons:
                std_rob = std_rows[std_rows['epsilon'] == eps]['robustness'].iloc[0] if len(std_rows[std_rows['epsilon'] == eps]) > 0 else 1.0
                adv_rob = adv_rows[adv_rows['epsilon'] == eps]['robustness'].max() if len(adv_rows[adv_rows['epsilon'] == eps]) > 0 else std_rob
                improvements.append(adv_rob - std_rob)
            
            best_improvement = max(improvements)
            worst_degradation = min(improvements)
            
            # Status
            if worst_degradation < -0.01:
                status = "\\textcolor{red}{Degrades}"
            elif best_improvement > 0.01:
                status = "\\textcolor{green}{Improves}"
            else:
                status = "Neutral"
            
            summary_rows.append({
                'attack': attack,
                'avg_robustness': avg_rob,
                'best_improvement': best_improvement,
                'worst_degradation': worst_degradation,
                'status': status
            })
    
    # Generate LaTeX table
    latex_table = """\\begin{table}[t]
\\centering
\\footnotesize
\\setlength{\\tabcolsep}{3pt}
\\caption{Summary: Adversarial Training Effectiveness at Training Epsilons}
\\label{tab:adversarial_effectiveness_summary}
\\begin{tabular}{lcccc}
\\toprule
Attack & Avg Robustness & Best Improvement & Worst Degradation & Status \\\\
\\midrule
"""
    
    for row in summary_rows:
        attack = row['attack']
        avg_rob = row['avg_robustness']
        best_imp = row['best_improvement']
        worst_deg = row['worst_degradation']
        status = row['status']
        
        latex_table += f"{attack} & {avg_rob:.4f} & {best_imp:.4f} & {worst_deg:.4f} & {status} \\\\\n"
    
    latex_table += """\\bottomrule
\\end{tabular}
\\vspace{0.1cm}
\\footnotesize
\\begin{minipage}{\\columnwidth}
\\textit{Note: Average robustness and improvement statistics at training epsilons ($\\epsilon \\in \\{0.25, 0.5, 1.0\\}$). Robustness is capped at 1.0 for interpretability. Best Improvement = maximum (Adversarial Robustness - Standard Robustness), Worst Degradation = minimum (Adversarial Robustness - Standard Robustness). Status indicates whether adversarial training overall helps (green), degrades (red), or is neutral. Key finding: Adversarial training maintains high robustness ($\\geq 0.99$) at training epsilons but shows slight degradation (0.003-0.033) relative to standard training.}
\\end{minipage}
\\end{table}
"""
    
    # Save table
    table_path = tables_dir / 'adversarial_effectiveness_summary.tex'
    with open(table_path, 'w') as f:
        f.write(latex_table)
    
    print(f" Table III saved to: {table_path}")
    print("\nTable preview:")
    print(latex_table[:500] + "...")
else:
    print("Robustness results not available. Run robustness evaluation first.")

### 5.4. Generate  Figures

Generate all figures needed for the paper in high resolution (300 DPI).

In [ ]:
# Generate Publication-Quality Figures
print("=" * 80)
print("GENERATING PUBLICATION-QUALITY FIGURES")
print("=" * 80)

# Set high DPI for all figures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

# Use publication-quality style
try:
    plt.style.use('seaborn-v0_8-paper')
except:
    try:
        plt.style.use('seaborn-paper')
    except:
        plt.style.use('default')

sns.set_palette("husl")

print("Figure settings configured for publication quality (300 DPI)")
print()

#### 5.4.1. Figure: Robustness vs Epsilon

In [ ]:
# Figure: Robustness vs Epsilon
print("Generating Figure: Robustness vs Epsilon...")

if 'robustness_df' in locals() and len(robustness_df) > 0:
    # Filter to training epsilons
    training_epsilons = [0.25, 0.5, 1.0]
    df_plot = robustness_df[robustness_df['epsilon'].isin(training_epsilons)].copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Robustness vs Epsilon: Standard vs Adversarially Trained Models', 
                 fontsize=14, fontweight='bold')
    
    attacks = ['A1', 'A2', 'A3', 'A4']
    epsilons = sorted(training_epsilons)
    
    for idx, attack in enumerate(attacks):
        ax = axes[idx // 2, idx % 2]
        
        # Standard model (Multi-Head Diversity)
        # Use 'training_type' instead of 'model_type', handle 'attack' vs 'attack_type'
        training_col = 'training_type' if 'training_type' in df_plot.columns else 'model_type'
        attack_col = 'attack' if 'attack' in df_plot.columns else 'attack_type'
        std_data = df_plot[(df_plot.get(training_col, pd.Series([True]*len(df_plot))) == 'standard') & 
                           (df_plot[attack_col].str.upper() == attack) &
                           (df_plot['model_name'] == 'Multi-Head Diversity')]
        if not std_data.empty:
            std_eps = std_data['epsilon'].values
            std_rob = std_data['robustness'].values
            ax.plot(std_eps, std_rob, 'o-', label='Standard', linewidth=2.5, 
                   markersize=8, color='#2E86AB', markerfacecolor='white', markeredgewidth=2)
        
        # Best adversarial model at each epsilon
        adv_eps = []
        adv_rob = []
        for eps in epsilons:
            training_col = 'training_type' if 'training_type' in df_plot.columns else 'model_type'
            attack_col = 'attack' if 'attack' in df_plot.columns else 'attack_type'
            adv_data = df_plot[(df_plot.get(training_col, pd.Series(['adversarial']*len(df_plot))) == 'adversarial') & 
                             (df_plot[attack_col].str.upper() == attack) & 
                             (df_plot['epsilon'] == eps)]
            if not adv_data.empty:
                best_rob_val = adv_data['robustness'].max()
                adv_eps.append(eps)
                adv_rob.append(best_rob_val)
        
        if adv_eps:
            ax.scatter(adv_eps, adv_rob, s=120, alpha=0.8, 
                      label='Best Adversarial', color='#A23B72', marker='s',
                      edgecolors='white', linewidths=2)
        
        ax.set_xlabel('Epsilon ($\\epsilon$)', fontsize=11, fontweight='bold')
        ax.set_ylabel('Robustness', fontsize=11, fontweight='bold')
        ax.set_title(f'{attack} Attack', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=10, framealpha=0.9)
        ax.set_ylim([0.9, 1.05])
        ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=1)
        ax.set_xticks(epsilons)
    
    plt.tight_layout()
    output_path = figures_dir / 'robustness_vs_epsilon_validation.pdf'
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f" Saved: {output_path}")
else:
    print("Robustness results not available.")

#### 5.4.2. Figure: Robustness Heatmap

In [ ]:
# Figure: Robustness Heatmap
print("Generating Figure: Robustness Heatmap...")

if 'robustness_df' in locals() and len(robustness_df) > 0:
    # Filter to training epsilons and Multi-Head Diversity
    training_epsilons = [0.25, 0.5, 1.0]
    df_plot = robustness_df[
        (robustness_df['epsilon'].isin(training_epsilons)) &
        (robustness_df['model_name'] == 'Multi-Head Diversity')
    ].copy()
    
    # Create pivot table for heatmap
    attacks = ['A1', 'A2', 'A3', 'A4']
    heatmap_data = []
    
    for attack in attacks:
        for eps in training_epsilons:
            training_col = 'training_type' if 'training_type' in df_plot.columns else 'model_type'
            attack_col = 'attack' if 'attack' in df_plot.columns else 'attack_type'
            row = df_plot[(df_plot[attack_col].str.upper() == attack) & 
                         (df_plot['epsilon'] == eps) &
                         (df_plot.get(training_col, pd.Series(['standard']*len(df_plot))) == 'standard')]
            if len(row) > 0:
                heatmap_data.append({
                    'Attack': attack,
                    'Epsilon': eps,
                    'Robustness': row['robustness'].iloc[0]
                })
    
    if heatmap_data:
        heatmap_df = pd.DataFrame(heatmap_data)
        pivot = heatmap_df.pivot(index='Attack', columns='Epsilon', values='Robustness')
        
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn', vmin=0.9, vmax=1.0,
                   cbar_kws={'label': 'Robustness'}, ax=ax, linewidths=0.5,
                   square=True, linecolor='white')
        ax.set_title('Robustness Heatmap: Multi-Head Diversity (Standard Training)', 
                    fontsize=13, fontweight='bold', pad=15)
        ax.set_xlabel('Epsilon ($\\epsilon$)', fontsize=11, fontweight='bold')
        ax.set_ylabel('Attack Type', fontsize=11, fontweight='bold')
        
        plt.tight_layout()
        output_path = figures_dir / 'robustness_heatmap_validation.pdf'
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print(f" Saved: {output_path}")
    else:
        print("No data for heatmap.")
else:
    print("Robustness results not available.")

#### 5.4.3. Figure: Improvement/Degradation Matrix

In [ ]:
# Figure: Improvement/Degradation Matrix (Line Plot)
print("Generating Figure: Improvement/Degradation Matrix...")

if 'robustness_df' in locals() and len(robustness_df) > 0:
    training_epsilons = [0.25, 0.5, 1.0]
    df_plot = robustness_df[robustness_df['epsilon'].isin(training_epsilons)].copy()
    
    # Calculate improvements (adversarial - standard) for each attack and epsilon
    improvements = []
    attacks = ['A1', 'A2', 'A3', 'A4']
    
    for attack in attacks:
        for eps in training_epsilons:
            training_col = 'training_type' if 'training_type' in df_plot.columns else 'model_type'
            attack_col = 'attack' if 'attack' in df_plot.columns else 'attack_type'
            std_row = df_plot[(df_plot.get(training_col, pd.Series(['standard']*len(df_plot))) == 'standard') & 
                             (df_plot[attack_col].str.upper() == attack) &
                             (df_plot['model_name'] == 'Multi-Head Diversity') &
                             (df_plot['epsilon'] == eps)]
            adv_rows = df_plot[(df_plot.get(training_col, pd.Series(['adversarial']*len(df_plot))) == 'adversarial') & 
                              (df_plot[attack_col].str.upper() == attack) &
                              (df_plot['epsilon'] == eps)]
            
            if len(std_row) > 0 and len(adv_rows) > 0:
                std_rob = std_row['robustness'].iloc[0]
                best_adv_rob = adv_rows['robustness'].max()
                improvement = best_adv_rob - std_rob
                improvements.append({
                    'Attack': attack,
                    'Epsilon': eps,
                    'Improvement': improvement
                })
    
    if improvements:
        imp_df = pd.DataFrame(improvements)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Plot line for each attack
        for attack in attacks:
            attack_data = imp_df[imp_df['Attack'] == attack]
            if len(attack_data) > 0:
                epsilons = sorted(attack_data['Epsilon'].values)
                improvements = [attack_data[attack_data['Epsilon'] == eps]['Improvement'].iloc[0] 
                               for eps in epsilons]
                color = '#A23B72' if attack == 'A1' else '#2E86AB' if attack == 'A2' else '#F18F01' if attack == 'A3' else '#C73E1D'
                marker = 'o' if attack == 'A1' else 's' if attack == 'A2' else '^' if attack == 'A3' else 'D'
                ax.plot(epsilons, improvements, marker=marker, linewidth=2.5, markersize=10,
                       label=f'{attack}', color=color, markerfacecolor='white', markeredgewidth=2)
        
        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, linewidth=1.5)
        ax.set_xlabel('Epsilon ($\\epsilon$)', fontsize=11, fontweight='bold')
        ax.set_ylabel('Improvement (Adversarial - Standard)', fontsize=11, fontweight='bold')
        ax.set_title('Adversarial Training Effectiveness: Improvement vs Degradation', 
                    fontsize=13, fontweight='bold')
        ax.legend(fontsize=10, framealpha=0.9, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_xticks(training_epsilons)
        
        # Add summary statistics
        avg_improvement = imp_df['Improvement'].mean()
        ax.text(0.02, 0.98, f'Average Improvement: {avg_improvement:.4f}', 
               transform=ax.transAxes, fontsize=10, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        plt.tight_layout()
        output_path = figures_dir / 'improvement_degradation_matrix_validation.pdf'
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print(f" Saved: {output_path}")
    else:
        print(" No data for improvement matrix.")
else:
    print(" Robustness results not available.")

#### 5.4.4. Save Time Series Plot 

The time series plot is already generated in section 4.2. This cell ensures it's saved to the figures directory.

In [ ]:
# Ensure time series plot is saved (if it was generated in section 4.2)
print("Checking for time series plot...")

timeseries_path = figures_dir / 'timeseries_predictions_validation.pdf'
if timeseries_path.exists():
    print(f" Time series plot already exists: {timeseries_path}")
else:
    print("Time series plot not found. Run section 4.2 to generate it.")
    print("The plot will be automatically saved when generated.")

### 5.5. Summary

All figures and tables have been generated and saved to:
- **Figures**: `paper/figures/`
- **Tables**: `paper/tables/`

These files are ready to be included in the LaTeX document.

In [ ]:
# Summary
print("=" * 80)
print("GENERATION COMPLETE")
print("=" * 80)
print(f"\n Tables saved to: {tables_dir}")
print(f" Figures saved to: {figures_dir}")
print("\nGenerated files:")
print("\nTables:")
for table_file in sorted(tables_dir.glob('*.tex')):
    print(f"  ✓ {table_file.name}")

print("\nFigures:")
for fig_file in sorted(figures_dir.glob('*.pdf')):
    if fig_file.name.endswith('_validation.pdf') or fig_file.name in ['timeseries_predictions_validation.pdf', 
                                                                        'feature_importance_attention.pdf',
                                                                        'figure_i_a4_predictions.pdf']:
        print(f" {fig_file.name}")

print("\nAll files are ready for LaTeX compilation!")
print("\nNote: Some figures (feature_importance_attention.pdf, figure_i_a4_predictions.pdf)")
print("may need to be generated separately using the dedicated scripts.")

In [ ]:
# Comprehensive Robustness Visualization
if 'robustness_results' in locals() and 'robustness_df' in locals() and len(robustness_results) > 0:
    # Filter to transformer models and training epsilons
    plot_data = robustness_df[
        (robustness_df['model_name'].isin(['Single-Head', 'Multi-Head', 'Multi-Head Diversity'])) &
        (robustness_df['epsilon'].isin(ATTACK_EPSILONS))
    ].copy()
    
    if len(plot_data) > 0:
        print("=" * 80)
        print("GENERATING COMPREHENSIVE ROBUSTNESS VISUALIZATIONS")
        print("=" * 80)
        
        # ============================================================
        # Plot 1: Robustness Bar Charts by Attack Type
        # ============================================================
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        for idx, attack_type in enumerate(ATTACK_TYPES):
            ax = axes[idx]
            attack_data = plot_data[plot_data['attack_type'] == attack_type]
            
            if len(attack_data) > 0:
                # Group by model and epsilon
                pivot_data = attack_data.pivot_table(
                    index='model_name', 
                    columns='epsilon', 
                    values='robustness', 
                    aggfunc='mean'
                )
                
                # Plot
                x_pos = np.arange(len(pivot_data.index))
                width = 0.25
                epsilons = sorted(attack_data['epsilon'].unique())
                colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
                
                for i, eps in enumerate(epsilons):
                    if eps in pivot_data.columns:
                        bars = ax.bar(x_pos + i*width, pivot_data[eps], width, 
                                     label=f'ε={eps}', alpha=0.8, color=colors[i % len(colors)])
                        # Add value labels on bars
                        for j, (bar, val) in enumerate(zip(bars, pivot_data[eps])):
                            height = bar.get_height()
                            ax.text(bar.get_x() + bar.get_width()/2., height,
                                   f'{val:.3f}', ha='center', va='bottom', fontsize=8)
                
                ax.set_xlabel('Model', fontsize=12)
                ax.set_ylabel('Robustness', fontsize=12)
                ax.set_title(f'{attack_type.upper()} Attack: Robustness by Model', fontsize=14, fontweight='bold')
                ax.set_xticks(x_pos + width)
                ax.set_xticklabels(pivot_data.index, rotation=45, ha='right')
                ax.legend(title='Epsilon', fontsize=10)
                ax.grid(True, alpha=0.3, axis='y', linestyle='--')
                ax.axhline(y=0.98, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Near-invariance (0.98)')
                ax.set_ylim([0.85, 1.01])
        
        plt.suptitle('Robustness Scores by Attack Type and Epsilon', fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()
        
        # ============================================================
        # Plot 2: Robustness vs Epsilon (Line Plot)
        # ============================================================
        fig, ax = plt.subplots(1, 1, figsize=(14, 7))
        
        model_colors = {'Single-Head': '#1f77b4', 'Multi-Head': '#ff7f0e', 'Multi-Head Diversity': '#2ca02c'}
        attack_markers = {'a1': 'o', 'a2': 's', 'a3': '^', 'a4': 'D'}
        attack_labels = {'a1': 'A1 (Measurement Error)', 'a2': 'A2 (Missingness)', 
                        'a3': 'A3 (Rank Manipulation)', 'a4': 'A4 (Regime Shift)'}
        
        for model_name in ['Single-Head', 'Multi-Head', 'Multi-Head Diversity']:
            model_data = plot_data[plot_data['model_name'] == model_name]
            if len(model_data) > 0:
                avg_robustness = model_data.groupby(['attack_type', 'epsilon'])['robustness'].mean().reset_index()
                
                for attack_type in ATTACK_TYPES:
                    attack_data = avg_robustness[avg_robustness['attack_type'] == attack_type]
                    if len(attack_data) > 0:
                        ax.plot(attack_data['epsilon'], attack_data['robustness'], 
                               marker=attack_markers[attack_type], 
                               label=f'{model_name} - {attack_labels[attack_type]}', 
                               linewidth=2.5, markersize=10, alpha=0.8,
                               color=model_colors[model_name])
        
        ax.set_xlabel('Epsilon (Attack Strength)', fontsize=13, fontweight='bold')
        ax.set_ylabel('Robustness Score', fontsize=13, fontweight='bold')
        ax.set_title('Robustness vs Epsilon: All Models and Attacks', fontsize=15, fontweight='bold')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, framealpha=0.9)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.axhline(y=0.98, color='green', linestyle='--', linewidth=2, alpha=0.7, 
                  label='Near-invariance threshold (0.98)')
        ax.set_ylim([0.85, 1.01])
        ax.set_xlim([0.2, 1.05])
        plt.tight_layout()
        plt.show()
        
        # ============================================================
        # Plot 3: Robustness Heatmap
        # ============================================================
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        for idx, model_name in enumerate(['Single-Head', 'Multi-Head', 'Multi-Head Diversity']):
            ax = axes[idx]
            model_data = plot_data[plot_data['model_name'] == model_name]
            
            if len(model_data) > 0:
                # Create pivot table for heatmap
                heatmap_data = model_data.pivot_table(
                    index='attack_type',
                    columns='epsilon',
                    values='robustness',
                    aggfunc='mean'
                )
                
                # Create heatmap
                im = ax.imshow(heatmap_data.values, cmap='RdYlGn', aspect='auto', 
                              vmin=0.85, vmax=1.0, interpolation='nearest')
                
                # Set ticks and labels
                ax.set_xticks(np.arange(len(heatmap_data.columns)))
                ax.set_yticks(np.arange(len(heatmap_data.index)))
                ax.set_xticklabels([f'ε={eps}' for eps in heatmap_data.columns])
                ax.set_yticklabels([at.upper() for at in heatmap_data.index])
                
                # Add text annotations
                for i in range(len(heatmap_data.index)):
                    for j in range(len(heatmap_data.columns)):
                        text = ax.text(j, i, f'{heatmap_data.iloc[i, j]:.3f}',
                                     ha="center", va="center", color="black", fontweight='bold', fontsize=10)
                
                ax.set_title(f'{model_name}\nRobustness Heatmap', fontsize=12, fontweight='bold')
                ax.set_xlabel('Epsilon', fontsize=11)
                ax.set_ylabel('Attack Type', fontsize=11)
        
        # Add colorbar
        cbar = fig.colorbar(im, ax=axes, orientation='horizontal', pad=0.1, aspect=40)
        cbar.set_label('Robustness Score', fontsize=12, fontweight='bold')
        plt.suptitle('Robustness Heatmaps: All Models', fontsize=15, fontweight='bold', y=1.05)
        plt.tight_layout()
        plt.show()
        
        # ============================================================
        # Plot 4: Delta RMSE (Degradation) Analysis
        # ============================================================
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        for idx, attack_type in enumerate(ATTACK_TYPES):
            ax = axes[idx]
            attack_data = plot_data[plot_data['attack_type'] == attack_type]
            
            if len(attack_data) > 0:
                # Group by model and epsilon
                pivot_delta = attack_data.pivot_table(
                    index='model_name',
                    columns='epsilon',
                    values='delta_rmse',
                    aggfunc='mean'
                )
                
                x_pos = np.arange(len(pivot_delta.index))
                width = 0.25
                epsilons = sorted(attack_data['epsilon'].unique())
                
                for i, eps in enumerate(epsilons):
                    if eps in pivot_delta.columns:
                        bars = ax.bar(x_pos + i*width, pivot_delta[eps] * 1000, width,
                                     label=f'ε={eps}', alpha=0.8)
                        # Add value labels
                        for j, (bar, val) in enumerate(zip(bars, pivot_delta[eps] * 1000)):
                            height = bar.get_height()
                            ax.text(bar.get_x() + bar.get_width()/2., height,
                                   f'{val:.2f}', ha='center', va='bottom' if height >= 0 else 'top', 
                                   fontsize=8)
                
                ax.set_xlabel('Model', fontsize=12)
                ax.set_ylabel('ΔRMSE (×1000)', fontsize=12)
                ax.set_title(f'{attack_type.upper()} Attack: RMSE Degradation', fontsize=14, fontweight='bold')
                ax.set_xticks(x_pos + width)
                ax.set_xticklabels(pivot_delta.index, rotation=45, ha='right')
                ax.legend(title='Epsilon', fontsize=10)
                ax.grid(True, alpha=0.3, axis='y', linestyle='--')
                ax.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
        
        plt.suptitle('RMSE Degradation Under Attacks (Lower is Better)', fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()
        
        # ============================================================
        # Plot 5: Standard vs Adversarially Trained Comparison
        # ============================================================
        if len(adversarial_models) > 0:
            # Separate standard and adversarial results
            standard_results = plot_data[plot_data['model_name'].isin(['Single-Head', 'Multi-Head', 'Multi-Head Diversity'])]
            adv_results = robustness_df[
                (~robustness_df['model_name'].isin(['OLS', 'Ridge', 'MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity'])) &
                (robustness_df['epsilon'].isin(ATTACK_EPSILONS))
            ].copy()
            
            if len(adv_results) > 0:
                adv_results['base_model'] = adv_results['model_name'].str.split('(').str[0].str.strip()
                
                fig, axes = plt.subplots(1, 3, figsize=(18, 5))
                
                for idx, base_model in enumerate(['Single-Head', 'Multi-Head', 'Multi-Head Diversity']):
                    ax = axes[idx]
                    
                    std_data = standard_results[standard_results['model_name'] == base_model]
                    adv_data = adv_results[adv_results['base_model'] == base_model]
                    
                    if len(std_data) > 0 and len(adv_data) > 0:
                        # Average robustness by attack type
                        std_avg = std_data.groupby('attack_type')['robustness'].mean()
                        adv_avg = adv_data.groupby('attack_type')['robustness'].mean()
                        
                        x = np.arange(len(ATTACK_TYPES))
                        width = 0.35
                        
                        bars1 = ax.bar(x - width/2, [std_avg.get(at, 0) for at in ATTACK_TYPES], 
                                      width, label='Standard Training', alpha=0.8, color='#1f77b4')
                        bars2 = ax.bar(x + width/2, [adv_avg.get(at, 0) for at in ATTACK_TYPES], 
                                      width, label='Adversarial Training', alpha=0.8, color='#ff7f0e')
                        
                        # Add value labels
                        for bars in [bars1, bars2]:
                            for bar in bars:
                                height = bar.get_height()
                                ax.text(bar.get_x() + bar.get_width()/2., height,
                                       f'{height:.3f}', ha='center', va='bottom', fontsize=9)
                        
                        ax.set_xlabel('Attack Type', fontsize=12)
                        ax.set_ylabel('Average Robustness', fontsize=12)
                        ax.set_title(f'{base_model}\nStandard vs Adversarial', fontsize=13, fontweight='bold')
                        ax.set_xticks(x)
                        ax.set_xticklabels([at.upper() for at in ATTACK_TYPES])
                        ax.legend(fontsize=10)
                        ax.grid(True, alpha=0.3, axis='y', linestyle='--')
                        ax.axhline(y=0.98, color='green', linestyle='--', linewidth=2, alpha=0.7)
                        ax.set_ylim([0.85, 1.01])
                
                plt.suptitle('Standard vs Adversarially Trained: Robustness Comparison', 
                            fontsize=15, fontweight='bold', y=1.02)
                plt.tight_layout()
                plt.show()
        
        # ============================================================
        # Plot 6: Summary Statistics
        # ============================================================
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Left: Average robustness by model
        ax = axes[0]
        model_avg_robustness = plot_data.groupby('model_name')['robustness'].mean().sort_values(ascending=False)
        bars = ax.barh(model_avg_robustness.index, model_avg_robustness.values, alpha=0.8, color='steelblue')
        for i, (bar, val) in enumerate(zip(bars, model_avg_robustness.values)):
            ax.text(val, i, f' {val:.4f}', va='center', fontsize=11, fontweight='bold')
        ax.set_xlabel('Average Robustness', fontsize=12, fontweight='bold')
        ax.set_title('Average Robustness Across All Attacks', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x', linestyle='--')
        ax.axvline(x=0.98, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Near-invariance (0.98)')
        ax.legend()
        
        # Right: Robustness by attack type
        ax = axes[1]
        attack_avg_robustness = plot_data.groupby('attack_type')['robustness'].mean().sort_values(ascending=False)
        colors_map = {'a1': '#1f77b4', 'a2': '#ff7f0e', 'a3': '#2ca02c', 'a4': '#d62728'}
        bars = ax.barh([at.upper() for at in attack_avg_robustness.index], 
                      attack_avg_robustness.values, 
                      alpha=0.8, 
                      color=[colors_map[at] for at in attack_avg_robustness.index])
        for i, (bar, val) in enumerate(zip(bars, attack_avg_robustness.values)):
            ax.text(val, i, f' {val:.4f}', va='center', fontsize=11, fontweight='bold')
        ax.set_xlabel('Average Robustness', fontsize=12, fontweight='bold')
        ax.set_title('Average Robustness by Attack Type', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x', linestyle='--')
        ax.axvline(x=0.98, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Near-invariance (0.98)')
        ax.legend()
        
        plt.suptitle('Robustness Summary Statistics', fontsize=15, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        
        print("=" * 80)
        print(" All robustness visualizations complete!")
        print("=" * 80)
        print(f"\nGenerated {6} comprehensive plots:")
        print("  1. Robustness Bar Charts by Attack Type (4 panels)")
        print("  2. Robustness vs Epsilon Line Plot")
        print("  3. Robustness Heatmaps (3 models)")
        print("  4. Delta RMSE Degradation Analysis (4 panels)")
        print("  5. Standard vs Adversarial Comparison (3 models)")
        print("  6. Summary Statistics (2 panels)")
    else:
        print("No data available for visualization. Run robustness evaluation first.")
else:
    print("No robustness results available. Run the 'Evaluate Models Under Adversarial Attacks' cell first.")

In [ ]:
def make_predictions(model, X, device='cpu', batch_size=128, is_sklearn=False):
    """Make predictions with a model (deterministic mode)."""
    if is_sklearn:
        # sklearn models
        return model.predict(X)
    
    # PyTorch models
    model.eval()
    
    # Ensure deterministic mode - disable dropout
    with torch.no_grad():
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.eval()
        
        predictions = []
        
        # Get num_features - handle both FeatureTokenTransformer and SingleHeadTransformer
        if hasattr(model, 'num_features'):
            num_features = model.num_features
        elif hasattr(model, 'model') and hasattr(model.model, 'num_features'):
            # SingleHeadTransformer wraps FeatureTokenTransformer in self.model
            num_features = model.model.num_features
        else:
            # Fallback: use input dimension
            num_features = X.shape[1]
        
        for i in range(0, len(X), batch_size):
            batch = X[i:i+batch_size]
            X_tensor = torch.FloatTensor(batch).to(device)
            
            # Handle padding if needed
            if X_tensor.shape[1] != num_features:
                if X_tensor.shape[1] < num_features:
                    padding = torch.zeros(X_tensor.shape[0], num_features - X_tensor.shape[1]).to(device)
                    X_tensor = torch.cat([X_tensor, padding], dim=1)
                else:
                    X_tensor = X_tensor[:, :num_features]
            
            pred = model(X_tensor)
            if isinstance(pred, tuple):
                pred = pred[0]
            predictions.append(pred.cpu().numpy())
    
    return np.concatenate(predictions, axis=0).flatten()

# Make predictions on validation set
print("Making predictions on validation set...")
predictions = {}

for name, model in models.items():
    is_sklearn = name in ['OLS', 'Ridge']
    pred = make_predictions(model, X_val_scaled, device, is_sklearn=is_sklearn)
    predictions[name] = pred.flatten()
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    r2 = r2_score(y_val, pred)
    
    print(f"{name}:")
    print(f"  RMSE: {rmse:.6f}")
    print(f"  R²: {r2:.6f}")
    print()
comparison_data = []
for name in ['OLS', 'Ridge', 'MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']:
    if name in training_history:
        comparison_data.append({
            'Model': name,
            'RMSE': training_history[name]['rmse'],
            'R²': training_history[name]['r2']
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('R²', ascending=False)

print("=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
ax1.barh(comparison_df['Model'], comparison_df['RMSE'])
ax1.set_xlabel('RMSE', fontsize=12)
ax1.set_title('RMSE Comparison (Lower is Better)', fontsize=14)
ax1.grid(True, alpha=0.3, axis='x')

# R² comparison
ax2.barh(comparison_df['Model'], comparison_df['R²'])
ax2.set_xlabel('R²', fontsize=12)
ax2.set_title('R² Comparison (Higher is Better)', fontsize=14)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 5.1. Save Results 

This is a fixed version that handles missing variables gracefully.

In [ ]:
# FIXED: Save all results to disk (handles missing variables)
import json
from datetime import datetime
import pickle

# Create output directory
output_dir = repo_root / 'notebooks' / 'saved_results'
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_prefix = output_dir / f'results_{timestamp}'

print("=" * 80)
print("SAVING ALL RESULTS (FIXED VERSION)")
print("=" * 80)
print(f"Output directory: {output_dir}")
print(f"Save prefix: {save_prefix}")
print()

# Initialize variables to track what was saved
robustness_path_saved = None
predictions_path_saved = None

# 1. Save standard models
print("1. Saving standard models...")
models_dir = save_prefix / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

if 'models' in locals():
    for name, model in models.items():
        if name in ['OLS', 'Ridge']:
            # Save sklearn models
            model_path = models_dir / f'{name.lower()}_model.pkl'
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
            print(f"  ✓ Saved {name} to {model_path}")
        else:
            # Save PyTorch models
            model_path = models_dir / f'{name.lower().replace(" ", "_")}_model.pt'
            torch.save({
                'model_state_dict': model.state_dict(),
                'model_name': name,
                'timestamp': timestamp
            }, model_path)
            print(f"  ✓ Saved {name} to {model_path}")
else:
    print("  ⚠ No models to save")

# 2. Save adversarially trained models
print("\n2. Saving adversarially trained models...")
if 'adversarial_models' in locals() and len(adversarial_models) > 0:
    adv_models_dir = models_dir / 'adversarial'
    adv_models_dir.mkdir(parents=True, exist_ok=True)
    
    for model_key, adv_model in adversarial_models.items():
        safe_name = model_key.replace(' ', '_').replace('(', '').replace(')', '').replace('ε=', 'eps').replace(',', '_')
        model_path = adv_models_dir / f'{safe_name}_model.pt'
        torch.save({
            'model_state_dict': adv_model.state_dict(),
            'model_name': model_key,
            'timestamp': timestamp
        }, model_path)
        print(f"  ✓ Saved {model_key} to {model_path}")
else:
    print("  ⚠ No adversarially trained models to save")

# 3. Save training history
print("\n3. Saving training history...")
if 'training_history' in locals() and len(training_history) > 0:
    training_history_path = save_prefix / 'training_history.json'
    training_history_json = {}
    for name, metrics in training_history.items():
        training_history_json[name] = {}
        for key, value in metrics.items():
            if isinstance(value, (np.ndarray, list)):
                training_history_json[name][key] = [float(v) if isinstance(v, (np.floating, np.integer)) else v for v in value]
            elif isinstance(value, (np.floating, np.integer)):
                training_history_json[name][key] = float(value)
            else:
                training_history_json[name][key] = value
    
    with open(training_history_path, 'w') as f:
        json.dump(training_history_json, f, indent=2)
    print(f"  ✓ Saved training history to {training_history_path}")
else:
    print("  ⚠ No training history to save")
    training_history_path = save_prefix / 'training_history.json'

# 4. Save adversarial training history
print("\n4. Saving adversarial training history...")
if 'adversarial_training_history' in locals() and len(adversarial_training_history) > 0:
    adv_history_path = save_prefix / 'adversarial_training_history.json'
    adv_history_json = {}
    for name, metrics in adversarial_training_history.items():
        adv_history_json[name] = {
            'rmse': float(metrics['rmse']),
            'r2': float(metrics['r2']),
            'history': {
                'train_loss': [float(v) for v in metrics['history']['train_loss']],
                'val_loss': [float(v) for v in metrics['history']['val_loss']],
                'train_clean_loss': [float(v) for v in metrics['history']['train_clean_loss']],
                'train_adv_loss': [float(v) for v in metrics['history']['train_adv_loss']]
            }
        }
    
    with open(adv_history_path, 'w') as f:
        json.dump(adv_history_json, f, indent=2)
    print(f"  ✓ Saved adversarial training history to {adv_history_path}")
else:
    print("  ⚠ No adversarial training history to save")
    adv_history_path = save_prefix / 'adversarial_training_history.json'

# 5. Save robustness results (FIXED: Check if variables exist)
print("\n5. Saving robustness results...")
if 'robustness_results' in locals() and 'robustness_df' in locals() and len(robustness_results) > 0:
    robustness_path = save_prefix / 'robustness_results.csv'
    robustness_df.to_csv(robustness_path, index=False)
    print(f"  ✓ Saved robustness results to {robustness_path}")
    
    # Also save as JSON
    robustness_json_path = save_prefix / 'robustness_results.json'
    robustness_json = robustness_df.to_dict('records')
    for record in robustness_json:
        for key, value in record.items():
            if isinstance(value, (np.floating, np.integer)):
                record[key] = float(value)
    with open(robustness_json_path, 'w') as f:
        json.dump(robustness_json, f, indent=2)
    print(f"  ✓ Saved robustness results (JSON) to {robustness_json_path}")
    robustness_path_saved = str(robustness_path)
else:
    print("  ⚠ No robustness results to save (run robustness evaluation first)")
    robustness_path_saved = None

# 6. Save predictions (FIXED: Check if variable exists)
print("\n6. Saving predictions...")
if 'predictions' in locals() and len(predictions) > 0:
    predictions_path = save_prefix / 'predictions.json'
    predictions_json = {}
    for name, pred in predictions.items():
        predictions_json[name] = {
            'predictions': pred.tolist() if isinstance(pred, np.ndarray) else pred,
            'actual': y_val.tolist() if isinstance(y_val, np.ndarray) else y_val.tolist()
        }
    with open(predictions_path, 'w') as f:
        json.dump(predictions_json, f, indent=2)
    print(f"  ✓ Saved predictions to {predictions_path}")
    predictions_path_saved = str(predictions_path)
else:
    print("  ⚠ No predictions to save (run prediction cells first)")
    predictions_path = save_prefix / 'predictions.json'
    predictions_path_saved = None

# 7. Save data info
print("\n7. Saving data information...")
if 'splitter' in locals() and 'X_train_scaled' in locals():
    data_info = {
        'train_period': f"{splitter.train_start} to {splitter.train_end}",
        'val_period': f"{splitter.val_start} to {splitter.val_end}",
        'train_samples': len(X_train_scaled),
        'val_samples': len(X_val_scaled),
        'num_features': X_train_scaled.shape[1],
        'target_column': y_train.name if hasattr(y_train, 'name') else 'returns_1d',
        'random_seed': RANDOM_SEED,
        'timestamp': timestamp
    }
    data_info_path = save_prefix / 'data_info.json'
    with open(data_info_path, 'w') as f:
        json.dump(data_info, f, indent=2)
    print(f"  ✓ Saved data info to {data_info_path}")
else:
    print("  ⚠ No data info to save")
    data_info_path = save_prefix / 'data_info.json'

# 8. Create summary file (FIXED: Handle missing variables)
print("\n8. Creating summary file...")
summary = {
    'timestamp': timestamp,
    'models_trained': list(models.keys()) if 'models' in locals() else [],
    'adversarial_models_trained': len(adversarial_models) if 'adversarial_models' in locals() else 0,
    'robustness_evaluations': len(robustness_results) if 'robustness_results' in locals() else 0,
    'training_history_models': list(training_history.keys()) if 'training_history' in locals() else [],
    'adversarial_training_history_models': list(adversarial_training_history.keys()) if 'adversarial_training_history' in locals() else [],
    'files_saved': {
        'models_dir': str(models_dir),
        'training_history': str(training_history_path),
        'robustness_results': robustness_path_saved,
        'predictions': predictions_path_saved,
        'data_info': str(data_info_path)
    }
}
summary_path = save_prefix / 'summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"  ✓ Saved summary to {summary_path}")

print("\n" + "=" * 80)
print("SAVE COMPLETE")
print("=" * 80)
print(f"\nAll results saved to: {save_prefix}")
print(f"\nTo load results later, use the 'Load Saved Results' cell below.")

uded

In [ ]:
# Save all results to disk
import json
from datetime import datetime
import pickle

# Create output directory
output_dir = repo_root / 'notebooks' / 'saved_results'
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_prefix = output_dir / f'results_{timestamp}'

print("=" * 80)
print("SAVING ALL RESULTS")
print("=" * 80)
print(f"Output directory: {output_dir}")
print(f"Save prefix: {save_prefix}")
print()

# 1. Save standard models
print("1. Saving standard models...")
models_dir = save_prefix / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

for name, model in models.items():
    if name in ['OLS', 'Ridge']:
        # Save sklearn models
        model_path = models_dir / f'{name.lower()}_model.pkl'
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"  ✓ Saved {name} to {model_path}")
    else:
        # Save PyTorch models
        model_path = models_dir / f'{name.lower().replace(" ", "_")}_model.pt'
        torch.save({
            'model_state_dict': model.state_dict(),
            'model_name': name,
            'timestamp': timestamp
        }, model_path)
        print(f"  ✓ Saved {name} to {model_path}")

# 2. Save adversarially trained models
print("\n2. Saving adversarially trained models...")
adv_models_dir = models_dir / 'adversarial'
adv_models_dir.mkdir(parents=True, exist_ok=True)

for model_key, adv_model in adversarial_models.items():
    safe_name = model_key.replace(' ', '_').replace('(', '').replace(')', '').replace('ε=', 'eps').replace(',', '_')
    model_path = adv_models_dir / f'{safe_name}_model.pt'
    torch.save({
        'model_state_dict': adv_model.state_dict(),
        'model_name': model_key,
        'timestamp': timestamp
    }, model_path)
    print(f"  ✓ Saved {model_key} to {model_path}")

# 3. Save training history
print("\n3. Saving training history...")
training_history_path = save_prefix / 'training_history.json'
# Convert numpy types to native Python types for JSON serialization
training_history_json = {}
for name, metrics in training_history.items():
    training_history_json[name] = {}
    for key, value in metrics.items():
        if isinstance(value, (np.ndarray, list)):
            training_history_json[name][key] = [float(v) if isinstance(v, (np.floating, np.integer)) else v for v in value]
        elif isinstance(value, (np.floating, np.integer)):
            training_history_json[name][key] = float(value)
        else:
            training_history_json[name][key] = value

with open(training_history_path, 'w') as f:
    json.dump(training_history_json, f, indent=2)
print(f" Saved training history to {training_history_path}")

# 4. Save adversarial training history
if adversarial_training_history:
    print("\n4. Saving adversarial training history...")
    adv_history_path = save_prefix / 'adversarial_training_history.json'
    adv_history_json = {}
    for name, metrics in adversarial_training_history.items():
        adv_history_json[name] = {
            'rmse': float(metrics['rmse']),
            'r2': float(metrics['r2']),
            'history': {
                'train_loss': [float(v) for v in metrics['history']['train_loss']],
                'val_loss': [float(v) for v in metrics['history']['val_loss']],
                'train_clean_loss': [float(v) for v in metrics['history']['train_clean_loss']],
                'train_adv_loss': [float(v) for v in metrics['history']['train_adv_loss']]
            }
        }
    
    with open(adv_history_path, 'w') as f:
        json.dump(adv_history_json, f, indent=2)
    print(f"  Saved adversarial training history to {adv_history_path}")

# 5. Save robustness results
print("\n5. Saving robustness results...")
if len(robustness_results) > 0:
    robustness_path = save_prefix / 'robustness_results.csv'
    robustness_df.to_csv(robustness_path, index=False)
    print(f"  Saved robustness results to {robustness_path}")
    
    # Also save as JSON for easier loading
    robustness_json_path = save_prefix / 'robustness_results.json'
    robustness_json = robustness_df.to_dict('records')
    # Convert numpy types
    for record in robustness_json:
        for key, value in record.items():
            if isinstance(value, (np.floating, np.integer)):
                record[key] = float(value)
    with open(robustness_json_path, 'w') as f:
        json.dump(robustness_json, f, indent=2)
    print(f"  Saved robustness results (JSON) to {robustness_json_path}")
else:
    print("  No robustness results to save")

# 6. Save predictions
print("\n6. Saving predictions...")
predictions_path = save_prefix / 'predictions.json'
predictions_json = {}
for name, pred in predictions.items():
    predictions_json[name] = {
        'predictions': pred.tolist() if isinstance(pred, np.ndarray) else pred,
        'actual': y_val.tolist() if isinstance(y_val, np.ndarray) else y_val.tolist()
    }
with open(predictions_path, 'w') as f:
    json.dump(predictions_json, f, indent=2)
print(f"  Saved predictions to {predictions_path}")

# 7. Save data info
print("\n7. Saving data information...")
data_info = {
    'train_period': f"{splitter.train_start} to {splitter.train_end}",
    'val_period': f"{splitter.val_start} to {splitter.val_end}",
    'train_samples': len(X_train_scaled),
    'val_samples': len(X_val_scaled),
    'num_features': X_train_scaled.shape[1],
    'target_column': y_train.name if hasattr(y_train, 'name') else 'returns_1d',
    'random_seed': RANDOM_SEED,
    'timestamp': timestamp
}
data_info_path = save_prefix / 'data_info.json'
with open(data_info_path, 'w') as f:
    json.dump(data_info, f, indent=2)
print(f"  Saved data info to {data_info_path}")

# 8. Create summary file
print("\n8. Creating summary file...")
summary = {
    'timestamp': timestamp,
    'models_trained': list(models.keys()),
    'adversarial_models_trained': len(adversarial_models),
    'robustness_evaluations': len(robustness_results),
    'training_history_models': list(training_history.keys()),
    'adversarial_training_history_models': list(adversarial_training_history.keys()),
    'files_saved': {
        'models_dir': str(models_dir),
        'training_history': str(training_history_path),
        'robustness_results': str(robustness_path) if len(robustness_results) > 0 else None,
        'predictions': str(predictions_path),
        'data_info': str(data_info_path)
    }
}
summary_path = save_prefix / 'summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"  Saved summary to {summary_path}")

print("\n" + "=" * 80)
print("SAVE COMPLETE")
print("=" * 80)
print(f"\nAll results saved to: {save_prefix}")
print(f"\nTo load results later, use the 'Load Saved Results' cell below.")

## 6. Load Saved Results


In [ ]:
# Load saved results (optional - use this instead of training from scratch)
# Uncomment and modify the path to load a specific saved run

# Option 1: Load the most recent saved results
saved_results_dir = repo_root / 'notebooks' / 'saved_results'
if saved_results_dir.exists():
    saved_runs = sorted([d for d in saved_results_dir.iterdir() if d.is_dir()], 
                       key=lambda x: x.name, reverse=True)
    if saved_runs:
        latest_run = saved_runs[0]
        print(f"Found {len(saved_runs)} saved run(s). Latest: {latest_run.name}")
        print(f"\nTo load, uncomment the code below and set: load_path = latest_run")
        print(f"Or specify a specific run: load_path = saved_results_dir / 'results_YYYYMMDD_HHMMSS'")
    else:
        print("No saved runs found.")
else:
    print("Saved results directory does not exist. Run the 'Save Results' cell first.")

# Option 2: Load a specific saved run (uncomment and modify)
# load_path = saved_results_dir / 'results_20240101_120000'  # Replace with your timestamp

# Uncomment below to load:
"""
if 'load_path' in locals() and load_path.exists():
    print("=" * 80)
    print("LOADING SAVED RESULTS")
    print("=" * 80)
    print(f"Loading from: {load_path}")
    
    # Load summary
    summary_path = load_path / 'summary.json'
    if summary_path.exists():
        with open(summary_path, 'r') as f:
            loaded_summary = json.load(f)
        print(f"\nLoaded run from: {loaded_summary['timestamp']}")
        print(f"Models: {loaded_summary['models_trained']}")
    
    # Load models
    models_dir = load_path / 'models'
    if models_dir.exists():
        print("\nLoading standard models...")
        for model_file in models_dir.glob('*.pt'):
            # Skip adversarial models directory
            if model_file.parent.name == 'adversarial':
                continue
            # Load PyTorch model
            checkpoint = torch.load(model_file, map_location=device, weights_only=False)
            model_name = checkpoint.get('model_name', model_file.stem)
            print(f"  Loading {model_name}...")
            # Note: You'll need to recreate the model architecture first
            # This is a placeholder - actual loading depends on model type
        
        for model_file in models_dir.glob('*.pkl'):
            with open(model_file, 'rb') as f:
                model = pickle.load(f)
            model_name = model_file.stem.replace('_model', '').upper()
            models[model_name] = model
            print(f"  ✓ Loaded {model_name}")
    
    # Load robustness results
    robustness_path = load_path / 'robustness_results.csv'
    if robustness_path.exists():
        robustness_df = pd.read_csv(robustness_path)
        robustness_results = robustness_df.to_dict('records')
        print(f"\n✓ Loaded {len(robustness_results)} robustness results")
    
    # Load training history
    training_history_path = load_path / 'training_history.json'
    if training_history_path.exists():
        with open(training_history_path, 'r') as f:
            training_history = json.load(f)
        print(f"✓ Loaded training history for {len(training_history)} models")
    
    print("\n✓ Load complete!")
else:
    print("No load_path specified or path does not exist.")
"""